In [1]:
import numpy as np
import random, math
import copy, sys

A = np.array([[math.sqrt(2),  -math.sqrt(3)], [math.sqrt(5), math.sqrt(7)]])
x_fin= np.array([math.pi*pow(2,10), -pow(2,5)*math.exp(1)])

b = np.dot(A, x_fin)
qubits = 3

print("Matrix A:")
print(A)
print("\nSolution vector x:")
print(x_fin)
print("\nVector b:")
print(b)

Matrix A:
[[ 1.41421356 -1.73205081]
 [ 2.23606798  2.64575131]]

Solution vector x:
[3216.99087728  -86.98501851]

Vector b:
[4700.17460023 6963.26955782]


In [2]:
Center = np.array([0,0])
Curr_Pow = 18
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution is ", x_fin)
print("Current Center is ", Center)
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  18
Solution is  [3216.99087728  -86.98501851]
Current Center is  [0 0]


# Matrix Q is
[[ 4.69388024e+11  1.92414535e+12  3.84829070e+12  0.00000000e+00
   0.00000000e+00  0.00000000e+00  4.76444507e+11  9.52889015e+11
   1.90577803e+12 -4.76444507e+11 -9.52889015e+11 -1.90577803e+12]
 [ 0.00000000e+00  1.90084872e+12  7.69658139e+12  0.00000000e+00
   0.00000000e+00  0.00000000e+00  9.52889015e+11  1.90577803e+12
   3.81155606e+12 -9.52889015e+11 -1.90577803e+12 -3.81155606e+12]
 [ 0.00000000e+00  0.00000000e+00  7.64998814e+12  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.90577803e+12  3.81155606e+12
   7.62311212e+12 -1.90577803e+12 -3.81155606e+12 -7.62311212e+12]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.92684651e+11
   1.92414535e+12  3.84829070e+12 -4.76444507e+11 -9.52889015e+11
  -1.90577803e+12  4.76444507e+11  9.52889015e+11  1.90577803e+12]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.94744198e+12  7.69658139e+12

In [3]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q01','q01'):469388023697.4509, ('q02','q02'):1900848721698.902, ('q03','q03'):7649988140613.805, ('q04','q04'):492684650606.54926, ('q05','q05'):1947441975517.0986, ('q06','q06'):7743174648250.198, ('q07','q07'):681803965608.8356, ('q08','q08'):2737997465937.6714, ('q09','q09'):10973553070755.344, ('q10','q10'):692585569111.1646, ('q11','q11'):2759560672942.329, ('q12','q12'):11016679484764.656}

quadratic = {('q01','q02'):1924145348608.0005, ('q01','q03'):3848290697216.001, ('q01','q07'):476444507257.5496, ('q01','q08'):952889014515.0992, ('q01','q09'):1905778029030.1985, ('q01','q10'):-476444507257.5496, ('q01','q11'):-952889014515.0992, ('q01','q12'):-1905778029030.1985, ('q02','q03'):7696581394432.002, ('q02','q07'):952889014515.0992, ('q02','q08'):1905778029030.1985, ('q02','q09'):3811556058060.397, ('q02','q10'):-952889014515.0992, ('q02','q11'):-1905778029030.1985, ('q02','q12'):-3811556058060.397, ('q03','q07'):1905778029030.1985, ('q03','q08'):3811556058060.397, ('q03','q09'):7623112116120.794, ('q03','q10'):-1905778029030.1985, ('q03','q11'):-3811556058060.397, ('q03','q12'):-7623112116120.794, ('q04','q05'):1924145348608.0005, ('q04','q06'):3848290697216.001, ('q04','q07'):-476444507257.5496, ('q04','q08'):-952889014515.0992, ('q04','q09'):-1905778029030.1985, ('q04','q10'):476444507257.5496, ('q04','q11'):952889014515.0992, ('q04','q12'):1905778029030.1985, ('q05','q06'):7696581394432.002, ('q05','q07'):-952889014515.0992, ('q05','q08'):-1905778029030.1985, ('q05','q09'):-3811556058060.397, ('q05','q10'):952889014515.0992, ('q05','q11'):1905778029030.1985, ('q05','q12'):3811556058060.397, ('q06','q07'):-1905778029030.1985, ('q06','q08'):-3811556058060.397, ('q06','q09'):-7623112116120.794, ('q06','q10'):1905778029030.1985, ('q06','q11'):3811556058060.397, ('q06','q12'):7623112116120.794, ('q07','q08'):2748779069440.0, ('q07','q09'):5497558138880.0, ('q08','q09'):10995116277760.0, ('q10','q11'):2748779069440.0, ('q10','q12'):5497558138880.0, ('q11','q12'):10995116277760.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 ... q12             energy num_oc. ...
0    0   0   0   0   0   0   0   0   0 ...   0                0.0     100 ...
67   0   0   0   0   0   0   0   0   0 ...   0                0.0       1 ...
1    1   0   0   0   0   0   0   0   0 ...   0  469388023697.4509      30 ...
2    0   0   0   1   0   0   0   0   0 ...   0 492684650606.54926      96 ...
72   0   0   0   1   0   0   0   0   0 ...   0 492684650606.54926       1 ...
3    0   0   0   0   0   0   1   0   0 ...   0  681803965608.8356      45 ...
4    1   0   0   0   0   0   0   0   0 ...   0  685529085551.0659      67 ...
5    0   0   0   0   0   0   0   0   0 ...   0  692585569111.1646      98 ...
6    0   0   0   1   0   0   1   0   0 ...   0  698044108957.8352      95 ...
69   0   0   0   1   0   0   1   0   0 ...   0  698044108957.8352       1 ...
7    1   0   0   1   0   0   0   0   0 ...   0  962072674304.0002      38 ...
8    0   0   0   0   0   0   1   0   0 ...   0    1374389534720.

In [4]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  15
Solution x is 3.21699087727594815078E+03
Current  x is 0.00000000000000000000E+00
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is 3.21699087727594815078E+03
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[ 6.06015359e+09  3.00647711e+10  6.01295421e+10  0.00000000e+00
   0.00000000e+00  0.00000000e+00  7.44444543e+09  1.48888909e+10
   2.97777817e+10 -7.44444543e+09 -1.48888909e+10 -2.97777817e+10]
 [ 0.00000000e+00  2.71526927e+10  1.20259084e+11  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.48888909e+10  2.97777817e+10
   5.95555634e+10 -1.48888909e+10 -2.97777817e+10 -5.95555634e+10]
 [ 0.00000000e+00  0.00000000e+00  1.14434928e+11  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.97777817e+10  5.95555634e+10
   1.19111127e+11 -2.97777817e+10 -5.95555634e+10 -1.19111127e+11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  8.97223195e+09
   3.00647711e+10  6.01295421e+10 

In [5]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  12
Solution x is 3.21699087727594815078E+03
Current  x is 0.00000000000000000000E+00
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is 3.21699087727594815078E+03
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[-6.45643857e+07  4.69762048e+08  9.39524096e+08  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.16319460e+08  2.32638920e+08
   4.65277839e+08 -1.16319460e+08 -2.32638920e+08 -4.65277839e+08]
 [ 0.00000000e+00  1.05752253e+08  1.87904819e+09  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.32638920e+08  4.65277839e+08
   9.30555678e+08 -2.32638920e+08 -4.65277839e+08 -9.30555678e+08]
 [ 0.00000000e+00  0.00000000e+00  1.15102860e+09  0.00000000e+00
   0.00000000e+00  0.00000000e+00  4.65277839e+08  9.30555678e+08
   1.86111136e+09 -4.65277839e+08 -9.30555678e+08 -1.86111136e+09]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.99445410e+08
   4.69762048e+08  9.39524096e+08 

In [6]:
linear = {('q01','q01'):-64564385.72733023, ('q02','q02'):105752252.54533958, ('q03','q03'):1151028601.0906792, ('q04','q04'):299445409.7273303, ('q05','q05'):833771843.4546607, ('q06','q06'):2607067782.9093213, ('q07','q07'):83540882.63805544, ('q08','q08'):502626085.2761109, ('q09','q09'):2347429450.5522223, ('q10','q10'):252003437.3619446, ('q11','q11'):839551194.7238892, ('q12','q12'):3021279669.4477787}

quadratic = {('q01','q02'):469762048.0000001, ('q01','q03'):939524096.0000002, ('q01','q07'):116319459.7796752, ('q01','q08'):232638919.5593504, ('q01','q09'):465277839.1187008, ('q01','q10'):-116319459.7796752, ('q01','q11'):-232638919.5593504, ('q01','q12'):-465277839.1187008, ('q02','q03'):1879048192.0000005, ('q02','q07'):232638919.5593504, ('q02','q08'):465277839.1187008, ('q02','q09'):930555678.2374016, ('q02','q10'):-232638919.5593504, ('q02','q11'):-465277839.1187008, ('q02','q12'):-930555678.2374016, ('q03','q07'):465277839.1187008, ('q03','q08'):930555678.2374016, ('q03','q09'):1861111356.4748032, ('q03','q10'):-465277839.1187008, ('q03','q11'):-930555678.2374016, ('q03','q12'):-1861111356.4748032, ('q04','q05'):469762048.0000001, ('q04','q06'):939524096.0000002, ('q04','q07'):-116319459.7796752, ('q04','q08'):-232638919.5593504, ('q04','q09'):-465277839.1187008, ('q04','q10'):116319459.7796752, ('q04','q11'):232638919.5593504, ('q04','q12'):465277839.1187008, ('q05','q06'):1879048192.0000005, ('q05','q07'):-232638919.5593504, ('q05','q08'):-465277839.1187008, ('q05','q09'):-930555678.2374016, ('q05','q10'):232638919.5593504, ('q05','q11'):465277839.1187008, ('q05','q12'):930555678.2374016, ('q06','q07'):-465277839.1187008, ('q06','q08'):-930555678.2374016, ('q06','q09'):-1861111356.4748032, ('q06','q10'):465277839.1187008, ('q06','q11'):930555678.2374016, ('q06','q12'):1861111356.4748032, ('q07','q08'):671088640.0, ('q07','q09'):1342177280.0, ('q08','q09'):2684354560.0, ('q10','q11'):671088640.0, ('q10','q12'):1342177280.0, ('q11','q12'):2684354560.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 ... q12            energy num_oc. ...
0    1   0   0   0   0   0   0   0   0 ...   0   -64564385.72733     165 ...
7    1   0   0   0   0   0   0   0   0 ...   0   -64564385.72733      16 ...
1    0   0   0   0   0   0   0   0   0 ...   0               0.0      69 ...
9    0   0   0   0   0   0   0   0   0 ...   0               0.0       8 ...
2    1   0   0   0   0   0   0   0   0 ...   0   71119591.854939      66 ...
15   1   0   0   0   0   0   0   0   0 ...   0   71119591.854939      12 ...
3    0   0   0   0   0   0   1   0   0 ...   0   83540882.638055      50 ...
77   0   0   0   0   0   0   1   0   0 ...   0   83540882.638055       1 ...
4    0   1   0   0   0   0   0   0   0 ...   0   105752252.54534      27 ...
5    0   1   0   0   0   0   0   0   0 ...   0  125116770.347934      31 ...
6    1   0   0   0   0   0   1   0   0 ...   0    135295956.6904      41 ...
8    1   0   0   1   0   0   0   0   0 ...   0       234881024.0      75 ...

In [7]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  12
Solution x is 3.21699087727594815078E+03
Current  x is 4.09600000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is -8.79009122724051849218E+02
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[ 1.70316638e+08  4.69762048e+08  9.39524096e+08  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.16319460e+08  2.32638920e+08
   4.65277839e+08 -1.16319460e+08 -2.32638920e+08 -4.65277839e+08]
 [ 0.00000000e+00  5.75514301e+08  1.87904819e+09  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.32638920e+08  4.65277839e+08
   9.30555678e+08 -2.32638920e+08 -4.65277839e+08 -9.30555678e+08]
 [ 0.00000000e+00  0.00000000e+00  2.09055270e+09  0.00000000e+00
   0.00000000e+00  0.00000000e+00  4.65277839e+08  9.30555678e+08
   1.86111136e+09 -4.65277839e+08 -9.30555678e+08 -1.86111136e+09]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.45643857e+07
   4.69762048e+08  9.39524096e+08

In [8]:
linear = {('q01','q01'):170316638.2726698, ('q02','q02'):575514300.5453397, ('q03','q03'):2090552697.0906796, ('q04','q04'):64564385.72733025, ('q05','q05'):364009795.45466053, ('q06','q06'):1667543686.9093213, ('q07','q07'):199860342.41773063, ('q08','q08'):735265004.8354613, ('q09','q09'):2812707289.6709228, ('q10','q10'):135683977.58226937, ('q11','q11'):606912275.1645387, ('q12','q12'):2556001830.3290777}

quadratic = {('q01','q02'):469762048.0000001, ('q01','q03'):939524096.0000002, ('q01','q07'):116319459.7796752, ('q01','q08'):232638919.5593504, ('q01','q09'):465277839.1187008, ('q01','q10'):-116319459.7796752, ('q01','q11'):-232638919.5593504, ('q01','q12'):-465277839.1187008, ('q02','q03'):1879048192.0000005, ('q02','q07'):232638919.5593504, ('q02','q08'):465277839.1187008, ('q02','q09'):930555678.2374016, ('q02','q10'):-232638919.5593504, ('q02','q11'):-465277839.1187008, ('q02','q12'):-930555678.2374016, ('q03','q07'):465277839.1187008, ('q03','q08'):930555678.2374016, ('q03','q09'):1861111356.4748032, ('q03','q10'):-465277839.1187008, ('q03','q11'):-930555678.2374016, ('q03','q12'):-1861111356.4748032, ('q04','q05'):469762048.0000001, ('q04','q06'):939524096.0000002, ('q04','q07'):-116319459.7796752, ('q04','q08'):-232638919.5593504, ('q04','q09'):-465277839.1187008, ('q04','q10'):116319459.7796752, ('q04','q11'):232638919.5593504, ('q04','q12'):465277839.1187008, ('q05','q06'):1879048192.0000005, ('q05','q07'):-232638919.5593504, ('q05','q08'):-465277839.1187008, ('q05','q09'):-930555678.2374016, ('q05','q10'):232638919.5593504, ('q05','q11'):465277839.1187008, ('q05','q12'):930555678.2374016, ('q06','q07'):-465277839.1187008, ('q06','q08'):-930555678.2374016, ('q06','q09'):-1861111356.4748032, ('q06','q10'):465277839.1187008, ('q06','q11'):930555678.2374016, ('q06','q12'):1861111356.4748032, ('q07','q08'):671088640.0, ('q07','q09'):1342177280.0, ('q08','q09'):2684354560.0, ('q10','q11'):671088640.0, ('q10','q12'):1342177280.0, ('q11','q12'):2684354560.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 ... q12            energy num_oc. ...
0    0   0   0   0   0   0   0   0   0 ...   0               0.0      79 ...
1    0   0   0   1   0   0   0   0   0 ...   0    64564385.72733      63 ...
53   0   0   0   1   0   0   0   0   0 ...   0    64564385.72733       1 ...
2    0   0   0   0   0   0   0   0   0 ...   0  135683977.582269      82 ...
3    0   0   0   1   0   0   1   0   0 ...   0  148105268.365386     151 ...
4    1   0   0   0   0   0   0   0   0 ...   0   170316638.27267      25 ...
5    1   0   0   0   0   0   0   0   0 ...   0  189681156.075264      50 ...
6    0   0   0   0   0   0   1   0   0 ...   0  199860342.417731      69 ...
7    1   0   0   1   0   0   0   0   0 ...   0       234881024.0      25 ...
63   1   0   0   1   0   0   0   0   0 ...   0       234881024.0       1 ...
8    0   0   0   1   0   0   0   0   0 ...   0  316567823.089275      27 ...
9    0   0   0   0   1   0   1   0   0 ...   0  331231218.313041      56 ...

In [9]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  9
Solution x is 3.21699087727594815078E+03
Current  x is 4.09600000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is -8.79009122724051849218E+02
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[  8444523.78408372   7340032.          14680064.
          0.                 0.                 0.
    1817491.55905743   3634983.11811485   7269966.2362297
   -1817491.55905743  -3634983.11811485  -7269966.2362297 ]
 [        0.          20559063.56816744  29360128.00000001
          0.                 0.                 0.
    3634983.11811485   7269966.2362297   14539932.4724594
   -3634983.11811485  -7269966.2362297  -14539932.4724594 ]
 [        0.                 0.          55798191.13633489
          0.                 0.                 0.
    7269966.2362297   14539932.4724594   29079864.9449188
   -7269966.2362297  -14539932.4724594  -29079864.9449188 ]
 [        0.         

In [10]:
linear = {('q01','q01'):8444523.784083722, ('q02','q02'):20559063.568167444, ('q03','q03'):55798191.13633489, ('q04','q04'):-4774507.784083721, ('q05','q05'):-5878999.5681674415, ('q06','q06'):2922064.863665119, ('q07','q07'):6632462.802216327, ('q08','q08'):18507805.604432654, ('q09','q09'):57987131.20886531, ('q10','q10'):-1389582.8022163268, ('q11','q11'):2463714.3955673464, ('q12','q12'):25898948.791134693}

quadratic = {('q01','q02'):7340032.000000002, ('q01','q03'):14680064.000000004, ('q01','q07'):1817491.559057425, ('q01','q08'):3634983.11811485, ('q01','q09'):7269966.2362297, ('q01','q10'):-1817491.559057425, ('q01','q11'):-3634983.11811485, ('q01','q12'):-7269966.2362297, ('q02','q03'):29360128.000000007, ('q02','q07'):3634983.11811485, ('q02','q08'):7269966.2362297, ('q02','q09'):14539932.4724594, ('q02','q10'):-3634983.11811485, ('q02','q11'):-7269966.2362297, ('q02','q12'):-14539932.4724594, ('q03','q07'):7269966.2362297, ('q03','q08'):14539932.4724594, ('q03','q09'):29079864.9449188, ('q03','q10'):-7269966.2362297, ('q03','q11'):-14539932.4724594, ('q03','q12'):-29079864.9449188, ('q04','q05'):7340032.000000002, ('q04','q06'):14680064.000000004, ('q04','q07'):-1817491.559057425, ('q04','q08'):-3634983.11811485, ('q04','q09'):-7269966.2362297, ('q04','q10'):1817491.559057425, ('q04','q11'):3634983.11811485, ('q04','q12'):7269966.2362297, ('q05','q06'):29360128.000000007, ('q05','q07'):-3634983.11811485, ('q05','q08'):-7269966.2362297, ('q05','q09'):-14539932.4724594, ('q05','q10'):3634983.11811485, ('q05','q11'):7269966.2362297, ('q05','q12'):14539932.4724594, ('q06','q07'):-7269966.2362297, ('q06','q08'):-14539932.4724594, ('q06','q09'):-29079864.9449188, ('q06','q10'):7269966.2362297, ('q06','q11'):14539932.4724594, ('q06','q12'):29079864.9449188, ('q07','q08'):10485760.0, ('q07','q09'):20971520.0, ('q08','q09'):41943040.0, ('q10','q11'):10485760.0, ('q10','q12'):20971520.0, ('q11','q12'):41943040.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12          energy num_oc. ...
0    0   0   0   0   1   0   0   0   0   0   0   0 -5878999.568167     171 ...
1    0   0   0   1   0   0   0   0   0   0   0   0 -4774507.784084     129 ...
2    0   0   0   1   0   0   0   0   0   1   0   0 -4346599.027243      87 ...
3    0   0   0   0   1   0   0   0   0   1   0   0 -3633599.252269      88 ...
54   0   0   0   0   1   0   0   0   0   1   0   0 -3633599.252269       1 ...
4    0   0   0   1   1   0   0   0   0   0   0   0 -3313475.352251      68 ...
5    0   0   0   0   1   0   1   0   0   0   0   0 -2881519.884066      85 ...
6    0   0   0   1   1   0   1   0   0   0   0   0 -2133487.227207      74 ...
57   0   0   0   1   1   0   1   0   0   0   0   0 -2133487.227207       1 ...
7    0   0   0   0   0   0   0   0   0   1   0   0 -1389582.802216      12 ...
8    0   0   0   0   1   0   1   0   0   1   0   0  -636119.568167      31 ...
9    0   0   0   0   0   0   0   0   0   0   0   0  

In [11]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  9
Solution x is 3.21699087727594815078E+03
Current  x is 3.07200000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is 1.44990877275948150782E+02
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[  1104491.78408372   7340032.          14680064.
          0.                 0.                 0.
    1817491.55905743   3634983.11811485   7269966.2362297
   -1817491.55905743  -3634983.11811485  -7269966.2362297 ]
 [        0.           5878999.56816744  29360128.00000001
          0.                 0.                 0.
    3634983.11811485   7269966.2362297   14539932.4724594
   -3634983.11811485  -7269966.2362297  -14539932.4724594 ]
 [        0.                 0.          26438063.13633489
          0.                 0.                 0.
    7269966.2362297   14539932.4724594   29079864.9449188
   -7269966.2362297  -14539932.4724594  -29079864.9449188 ]
 [        0.          

In [12]:
linear = {('q01','q01'):1104491.7840837217, ('q02','q02'):5878999.568167443, ('q03','q03'):26438063.136334892, ('q04','q04'):2565524.2159162792, ('q05','q05'):8801064.43183256, ('q06','q06'):32282192.86366512, ('q07','q07'):2997479.684101477, ('q08','q08'):11237839.36820295, ('q09','q09'):43447198.73640591, ('q10','q10'):2245400.3158985237, ('q11','q11'):9733680.63179705, ('q12','q12'):40438881.2635941}

quadratic = {('q01','q02'):7340032.000000002, ('q01','q03'):14680064.000000004, ('q01','q07'):1817491.559057425, ('q01','q08'):3634983.11811485, ('q01','q09'):7269966.2362297, ('q01','q10'):-1817491.559057425, ('q01','q11'):-3634983.11811485, ('q01','q12'):-7269966.2362297, ('q02','q03'):29360128.000000007, ('q02','q07'):3634983.11811485, ('q02','q08'):7269966.2362297, ('q02','q09'):14539932.4724594, ('q02','q10'):-3634983.11811485, ('q02','q11'):-7269966.2362297, ('q02','q12'):-14539932.4724594, ('q03','q07'):7269966.2362297, ('q03','q08'):14539932.4724594, ('q03','q09'):29079864.9449188, ('q03','q10'):-7269966.2362297, ('q03','q11'):-14539932.4724594, ('q03','q12'):-29079864.9449188, ('q04','q05'):7340032.000000002, ('q04','q06'):14680064.000000004, ('q04','q07'):-1817491.559057425, ('q04','q08'):-3634983.11811485, ('q04','q09'):-7269966.2362297, ('q04','q10'):1817491.559057425, ('q04','q11'):3634983.11811485, ('q04','q12'):7269966.2362297, ('q05','q06'):29360128.000000007, ('q05','q07'):-3634983.11811485, ('q05','q08'):-7269966.2362297, ('q05','q09'):-14539932.4724594, ('q05','q10'):3634983.11811485, ('q05','q11'):7269966.2362297, ('q05','q12'):14539932.4724594, ('q06','q07'):-7269966.2362297, ('q06','q08'):-14539932.4724594, ('q06','q09'):-29079864.9449188, ('q06','q10'):7269966.2362297, ('q06','q11'):14539932.4724594, ('q06','q12'):29079864.9449188, ('q07','q08'):10485760.0, ('q07','q09'):20971520.0, ('q08','q09'):41943040.0, ('q10','q11'):10485760.0, ('q10','q12'):20971520.0, ('q11','q12'):41943040.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12          energy num_oc. ...
0    0   0   0   0   0   0   0   0   0   0   0   0             0.0     100 ...
72   0   0   0   0   0   0   0   0   0   0   0   0             0.0       1 ...
1    1   0   0   0   0   0   0   0   0   0   0   0  1104491.784084      55 ...
2    1   0   0   0   0   0   0   0   0   1   0   0  1532400.540925      54 ...
3    0   0   0   0   0   0   0   0   0   1   0   0  2245400.315899      43 ...
4    0   0   0   1   0   0   0   0   0   0   0   0  2565524.215916      80 ...
5    0   0   0   0   0   0   1   0   0   0   0   0  2997479.684101      70 ...
6    1   0   0   1   0   0   0   0   0   0   0   0       3670016.0      54 ...
7    0   0   0   1   0   0   1   0   0   0   0   0   3745512.34096     115 ...
8    0   1   0   0   0   0   0   0   0   1   0   0  4489416.765951      16 ...
9    0   0   0   0   0   0   1   0   0   1   0   0       5242880.0      23 ...
10   0   1   0   0   0   0   0   0   0   0   0   0  

In [13]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  6
Solution x is 3.21699087727594815078E+03
Current  x is 3.07200000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is 1.44990877275948150782E+02
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[ -62642.52698953  114688.          229376.               0.
        0.               0.           28398.30561027   56796.61122054
   113593.22244109  -28398.30561027  -56796.61122054 -113593.22244109]
 [      0.          -67941.05397907  458752.               0.
        0.               0.           56796.61122054  113593.22244109
   227186.44488218  -56796.61122054 -113593.22244109 -227186.44488218]
 [      0.               0.           93493.89204186       0.
        0.               0.          113593.22244109  227186.44488218
   454372.88976436 -113593.22244109 -227186.44488218 -454372.88976436]
 [      0.               0.               0.          119986.52698953
   114688.         

In [14]:
linear = {('q01','q01'):-62642.52698953483, ('q02','q02'):-67941.05397906966, ('q03','q03'):93493.89204186074, ('q04','q04'):119986.52698953483, ('q05','q05'):297317.0539790697, ('q06','q06'):824010.1079581395, ('q07','q07'):87964.96051268453, ('q08','q08'):257849.92102536905, ('q09','q09'):843379.8420507382, ('q10','q10'):-6044.960512684531, ('q11','q11'):69830.07897463095, ('q12','q12'):467340.1579492619}

quadratic = {('q01','q02'):114688.00000000003, ('q01','q03'):229376.00000000006, ('q01','q07'):28398.305610272266, ('q01','q08'):56796.61122054453, ('q01','q09'):113593.22244108906, ('q01','q10'):-28398.305610272266, ('q01','q11'):-56796.61122054453, ('q01','q12'):-113593.22244108906, ('q02','q03'):458752.0000000001, ('q02','q07'):56796.61122054453, ('q02','q08'):113593.22244108906, ('q02','q09'):227186.44488217813, ('q02','q10'):-56796.61122054453, ('q02','q11'):-113593.22244108906, ('q02','q12'):-227186.44488217813, ('q03','q07'):113593.22244108906, ('q03','q08'):227186.44488217813, ('q03','q09'):454372.88976435625, ('q03','q10'):-113593.22244108906, ('q03','q11'):-227186.44488217813, ('q03','q12'):-454372.88976435625, ('q04','q05'):114688.00000000003, ('q04','q06'):229376.00000000006, ('q04','q07'):-28398.305610272266, ('q04','q08'):-56796.61122054453, ('q04','q09'):-113593.22244108906, ('q04','q10'):28398.305610272266, ('q04','q11'):56796.61122054453, ('q04','q12'):113593.22244108906, ('q05','q06'):458752.0000000001, ('q05','q07'):-56796.61122054453, ('q05','q08'):-113593.22244108906, ('q05','q09'):-227186.44488217813, ('q05','q10'):56796.61122054453, ('q05','q11'):113593.22244108906, ('q05','q12'):227186.44488217813, ('q06','q07'):-113593.22244108906, ('q06','q08'):-227186.44488217813, ('q06','q09'):-454372.88976435625, ('q06','q10'):113593.22244108906, ('q06','q11'):227186.44488217813, ('q06','q12'):454372.88976435625, ('q07','q08'):163840.0, ('q07','q09'):327680.0, ('q08','q09'):655360.0, ('q10','q11'):163840.0, ('q10','q12'):327680.0, ('q11','q12'):655360.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12         energy num_oc. ...
0    0   1   0   0   0   0   0   0   0   1   0   0 -130782.625712     135 ...
1    1   1   0   0   0   0   0   0   0   0   1   0 -116455.335656     174 ...
2    0   1   0   0   0   0   0   0   0   0   1   0 -111704.197446     149 ...
3    1   1   0   0   0   0   0   0   0   1   0   0 -107135.458312      68 ...
4    1   0   0   0   0   0   0   0   0   1   0   0  -97085.793112      64 ...
5    0   1   0   0   0   0   0   0   0   0   0   0  -67941.053979      33 ...
6    0   0   1   0   0   0   0   0   0   0   1   0  -63862.473866      39 ...
45   0   0   1   0   0   0   0   0   0   0   1   0  -63862.473866       1 ...
7    1   0   0   0   0   0   0   0   0   0   0   0   -62642.52699      28 ...
8    1   0   0   0   0   0   0   0   0   0   1   0  -49609.059235      41 ...
9    1   1   0   0   0   0   0   0   0   1   1   0  -43855.212999      47 ...
10   0   0   1   0   0   0   0   0   0   1   0   0  -26144.29091

In [15]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  6
Solution x is 3.21699087727594815078E+03
Current  x is 3.20000000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -6.40000000000000000000E+01
Previous error of x is 1.69908772759481507819E+01
Previous error of y is -2.29850185106894429055E+01


# Matrix Q is
[[  23647.16740019  114688.          229376.               0.
        0.               0.           28398.30561027   56796.61122054
   113593.22244109  -28398.30561027  -56796.61122054 -113593.22244109]
 [      0.          104638.33480039  458752.               0.
        0.               0.           56796.61122054  113593.22244109
   227186.44488218  -56796.61122054 -113593.22244109 -227186.44488218]
 [      0.               0.          438652.66960077       0.
        0.               0.          113593.22244109  227186.44488218
   454372.88976436 -113593.22244109 -227186.44488218 -454372.88976436]
 [      0.               0.               0.           33696.83259981
   114688.        

In [16]:
linear = {('q01','q01'):23647.167400192862, ('q02','q02'):104638.33480038572, ('q03','q03'):438652.6696007715, ('q04','q04'):33696.83259980715, ('q05','q05'):124737.6651996143, ('q06','q06'):478851.33039922867, ('q07','q07'):62841.571733229015, ('q08','q08'):207603.14346645802, ('q09','q09'):742886.286932916, ('q10','q10'):19078.428266770992, ('q11','q11'):120076.85653354197, ('q12','q12'):567833.713067084}

quadratic = {('q01','q02'):114688.00000000003, ('q01','q03'):229376.00000000006, ('q01','q07'):28398.305610272266, ('q01','q08'):56796.61122054453, ('q01','q09'):113593.22244108906, ('q01','q10'):-28398.305610272266, ('q01','q11'):-56796.61122054453, ('q01','q12'):-113593.22244108906, ('q02','q03'):458752.0000000001, ('q02','q07'):56796.61122054453, ('q02','q08'):113593.22244108906, ('q02','q09'):227186.44488217813, ('q02','q10'):-56796.61122054453, ('q02','q11'):-113593.22244108906, ('q02','q12'):-227186.44488217813, ('q03','q07'):113593.22244108906, ('q03','q08'):227186.44488217813, ('q03','q09'):454372.88976435625, ('q03','q10'):-113593.22244108906, ('q03','q11'):-227186.44488217813, ('q03','q12'):-454372.88976435625, ('q04','q05'):114688.00000000003, ('q04','q06'):229376.00000000006, ('q04','q07'):-28398.305610272266, ('q04','q08'):-56796.61122054453, ('q04','q09'):-113593.22244108906, ('q04','q10'):28398.305610272266, ('q04','q11'):56796.61122054453, ('q04','q12'):113593.22244108906, ('q05','q06'):458752.0000000001, ('q05','q07'):-56796.61122054453, ('q05','q08'):-113593.22244108906, ('q05','q09'):-227186.44488217813, ('q05','q10'):56796.61122054453, ('q05','q11'):113593.22244108906, ('q05','q12'):227186.44488217813, ('q06','q07'):-113593.22244108906, ('q06','q08'):-227186.44488217813, ('q06','q09'):-454372.88976435625, ('q06','q10'):113593.22244108906, ('q06','q11'):227186.44488217813, ('q06','q12'):454372.88976435625, ('q07','q08'):163840.0, ('q07','q09'):327680.0, ('q08','q09'):655360.0, ('q10','q11'):163840.0, ('q10','q12'):327680.0, ('q11','q12'):655360.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12        energy num_oc. ...
0    0   0   0   0   0   0   0   0   0   0   0   0           0.0     216 ...
63   0   0   0   0   0   0   0   0   0   0   0   0           0.0       1 ...
1    1   0   0   0   0   0   0   0   0   1   0   0  14327.290057      87 ...
66   1   0   0   0   0   0   0   0   0   1   0   0  14327.290057       1 ...
2    0   0   0   0   0   0   0   0   0   1   0   0  19078.428267     117 ...
56   0   0   0   0   0   0   0   0   0   1   0   0  19078.428267       1 ...
62   0   0   0   0   0   0   0   0   0   1   0   0  19078.428267       1 ...
3    1   0   0   0   0   0   0   0   0   0   0   0    23647.1674      65 ...
52   1   0   0   0   0   0   0   0   0   0   0   0    23647.1674       1 ...
64   1   0   0   0   0   0   0   0   0   0   0   0    23647.1674       1 ...
65   1   0   0   0   0   0   0   0   0   0   0   0    23647.1674       1 ...
4    0   0   0   1   0   0   0   0   0   0   0   0    33696.8326      58 ...

In [17]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  3
Solution x is 3.21699087727594815078E+03
Current  x is 3.20000000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -6.40000000000000000000E+01
Previous error of x is 1.69908772759481507819E+01
Previous error of y is -2.29850185106894429055E+01


# Matrix Q is
[[ -180.10407498  1792.          3584.             0.
      0.             0.           443.72352516   887.44705032
   1774.89410064  -443.72352516  -887.44705032 -1774.89410064]
 [    0.           535.79185005  7168.             0.
      0.             0.           887.44705032  1774.89410064
   3549.78820128  -887.44705032 -1774.89410064 -3549.78820128]
 [    0.             0.          4655.5837001      0.
      0.             0.          1774.89410064  3549.78820128
   7099.57640257 -1774.89410064 -3549.78820128 -7099.57640257]
 [    0.             0.             0.          1076.10407498
   1792.          3584.          -443.72352516  -887.44705032
  -1774.89410064   443.72352516   88

In [18]:
linear = {('q01','q01'):-180.1040749758929, ('q02','q02'):535.7918500482144, ('q03','q03'):4655.583700096429, ('q04','q04'):1076.104074975893, ('q05','q05'):3048.208149951786, ('q06','q06'):9680.416299903573, ('q07','q07'):3375.196466653627, ('q08','q08'):8030.392933307252, ('q09','q09'):21180.785866614508, ('q10','q10'):-2095.1964666536264, ('q11','q11'):-2910.392933307253, ('q12','q12'):-700.7858666145053}

quadratic = {('q01','q02'):1792.0000000000005, ('q01','q03'):3584.000000000001, ('q01','q07'):443.72352516050415, ('q01','q08'):887.4470503210083, ('q01','q09'):1774.8941006420166, ('q01','q10'):-443.72352516050415, ('q01','q11'):-887.4470503210083, ('q01','q12'):-1774.8941006420166, ('q02','q03'):7168.000000000002, ('q02','q07'):887.4470503210083, ('q02','q08'):1774.8941006420166, ('q02','q09'):3549.788201284033, ('q02','q10'):-887.4470503210083, ('q02','q11'):-1774.8941006420166, ('q02','q12'):-3549.788201284033, ('q03','q07'):1774.8941006420166, ('q03','q08'):3549.788201284033, ('q03','q09'):7099.576402568066, ('q03','q10'):-1774.8941006420166, ('q03','q11'):-3549.788201284033, ('q03','q12'):-7099.576402568066, ('q04','q05'):1792.0000000000005, ('q04','q06'):3584.000000000001, ('q04','q07'):-443.72352516050415, ('q04','q08'):-887.4470503210083, ('q04','q09'):-1774.8941006420166, ('q04','q10'):443.72352516050415, ('q04','q11'):887.4470503210083, ('q04','q12'):1774.8941006420166, ('q05','q06'):7168.000000000002, ('q05','q07'):-887.4470503210083, ('q05','q08'):-1774.8941006420166, ('q05','q09'):-3549.788201284033, ('q05','q10'):887.4470503210083, ('q05','q11'):1774.8941006420166, ('q05','q12'):3549.788201284033, ('q06','q07'):-1774.8941006420166, ('q06','q08'):-3549.788201284033, ('q06','q09'):-7099.576402568066, ('q06','q10'):1774.8941006420166, ('q06','q11'):3549.788201284033, ('q06','q12'):7099.576402568066, ('q07','q08'):2560.0, ('q07','q09'):5120.0, ('q08','q09'):10240.0, ('q10','q11'):2560.0, ('q10','q12'):5120.0, ('q11','q12'):10240.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12       energy num_oc. ...
0    0   1   0   0   0   0   0   0   0   1   1   0 -4572.138701     194 ...
1    1   1   0   0   0   0   0   0   0   1   1   0 -4291.413351     203 ...
2    0   1   0   0   0   0   0   0   0   0   1   0 -4149.495184      95 ...
3    1   0   0   0   0   0   0   0   0   0   1   0 -3977.944059     104 ...
4    1   0   0   0   0   0   0   0   0   1   1   0  -3956.86405     110 ...
5    1   1   0   0   0   0   0   0   0   0   0   1 -3877.780393      74 ...
6    0   1   0   0   0   0   0   0   0   0   0   1 -3714.782218      25 ...
7    1   1   0   0   0   0   0   0   0   0   1   0 -3425.046309      52 ...
8    0   0   1   0   0   0   0   0   0   0   0   1 -3144.778569      11 ...
9    0   0   1   0   0   0   0   0   0   1   1   0 -3114.688002      40 ...
10   0   0   0   0   0   0   0   0   0   0   1   0 -2910.392933       9 ...
11   1   0   0   0   0   0   0   0   0   1   0   0 -2719.024067      11 ...
12   1   0  

In [19]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  3
Solution x is 3.21699087727594815078E+03
Current  x is 3.21600000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.80000000000000000000E+01
Previous error of x is 9.90877275948150781915E-01
Previous error of y is 1.01498148931055709454E+00


# Matrix Q is
[[  280.72534954  1792.          3584.             0.
      0.             0.           443.72352516   887.44705032
   1774.89410064  -443.72352516  -887.44705032 -1774.89410064]
 [    0.          1457.45069909  7168.             0.
      0.             0.           887.44705032  1774.89410064
   3549.78820128  -887.44705032 -1774.89410064 -3549.78820128]
 [    0.             0.          6498.90139817     0.
      0.             0.          1774.89410064  3549.78820128
   7099.57640257 -1774.89410064 -3549.78820128 -7099.57640257]
 [    0.             0.             0.           615.27465046
   1792.          3584.          -443.72352516  -887.44705032
  -1774.89410064   443.72352516   887

In [20]:
linear = {('q01','q01'):280.7253495426131, ('q02','q02'):1457.4506990852265, ('q03','q03'):6498.901398170454, ('q04','q04'):615.274650457387, ('q05','q05'):2126.5493009147744, ('q06','q06'):7837.098601829549, ('q07','q07'):422.64351697464605, ('q08','q08'):2125.287033949292, ('q09','q09'):9370.574067898584, ('q10','q10'):857.356483025354, ('q11','q11'):2994.712966050708, ('q12','q12'):11109.425932101418}

quadratic = {('q01','q02'):1792.0000000000005, ('q01','q03'):3584.000000000001, ('q01','q07'):443.72352516050415, ('q01','q08'):887.4470503210083, ('q01','q09'):1774.8941006420166, ('q01','q10'):-443.72352516050415, ('q01','q11'):-887.4470503210083, ('q01','q12'):-1774.8941006420166, ('q02','q03'):7168.000000000002, ('q02','q07'):887.4470503210083, ('q02','q08'):1774.8941006420166, ('q02','q09'):3549.788201284033, ('q02','q10'):-887.4470503210083, ('q02','q11'):-1774.8941006420166, ('q02','q12'):-3549.788201284033, ('q03','q07'):1774.8941006420166, ('q03','q08'):3549.788201284033, ('q03','q09'):7099.576402568066, ('q03','q10'):-1774.8941006420166, ('q03','q11'):-3549.788201284033, ('q03','q12'):-7099.576402568066, ('q04','q05'):1792.0000000000005, ('q04','q06'):3584.000000000001, ('q04','q07'):-443.72352516050415, ('q04','q08'):-887.4470503210083, ('q04','q09'):-1774.8941006420166, ('q04','q10'):443.72352516050415, ('q04','q11'):887.4470503210083, ('q04','q12'):1774.8941006420166, ('q05','q06'):7168.000000000002, ('q05','q07'):-887.4470503210083, ('q05','q08'):-1774.8941006420166, ('q05','q09'):-3549.788201284033, ('q05','q10'):887.4470503210083, ('q05','q11'):1774.8941006420166, ('q05','q12'):3549.788201284033, ('q06','q07'):-1774.8941006420166, ('q06','q08'):-3549.788201284033, ('q06','q09'):-7099.576402568066, ('q06','q10'):1774.8941006420166, ('q06','q11'):3549.788201284033, ('q06','q12'):7099.576402568066, ('q07','q08'):2560.0, ('q07','q09'):5120.0, ('q08','q09'):10240.0, ('q10','q11'):2560.0, ('q10','q12'):5120.0, ('q11','q12'):10240.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12       energy num_oc. ...
0    0   0   0   0   0   0   0   0   0   0   0   0          0.0     134 ...
23   0   0   0   0   0   0   0   0   0   0   0   0          0.0       1 ...
1    1   0   0   0   0   0   0   0   0   0   0   0    280.72535     100 ...
2    0   0   0   0   0   0   1   0   0   0   0   0   422.643517      51 ...
3    0   0   0   1   0   0   1   0   0   0   0   0   594.194642      88 ...
4    0   0   0   1   0   0   0   0   0   0   0   0    615.27465     113 ...
5    1   0   0   0   0   0   0   0   0   1   0   0   694.358307      55 ...
6    0   0   0   0   0   0   0   0   0   1   0   0   857.356483      47 ...
7    1   0   0   1   0   0   0   0   0   0   0   0        896.0      66 ...
59   1   0   0   1   0   0   0   0   0   0   0   0        896.0       1 ...
8    1   0   0   0   0   0   1   0   0   0   0   0  1147.092392      25 ...
60   1   0   0   0   0   0   1   0   0   0   0   0  1147.092392       1 ...
9    0   0  

In [21]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  0
Solution x is 3.21699087727594815078E+03
Current  x is 3.21600000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.80000000000000000000E+01
Previous error of x is 9.90877275948150781915E-01
Previous error of y is 1.01498148931055709454E+00


# Matrix Q is
[[ -13.90933131   28.           56.            0.            0.
     0.            6.93318008   13.86636016   27.73272032   -6.93318008
   -13.86636016  -27.73272032]
 [   0.          -13.81866261  112.            0.            0.
     0.           13.86636016   27.73272032   55.46544065  -13.86636016
   -27.73272032  -55.46544065]
 [   0.            0.           28.36267477    0.            0.
     0.           27.73272032   55.46544065  110.93088129  -27.73272032
   -55.46544065 -110.93088129]
 [   0.            0.            0.           27.90933131   28.
    56.           -6.93318008  -13.86636016  -27.73272032    6.93318008
    13.86636016   27.73272032]
 [   0.            0.         

In [22]:
linear = {('q01','q01'):-13.909331307173366, ('q02','q02'):-13.818662614346728, ('q03','q03'):28.362674771306544, ('q04','q04'):27.909331307173368, ('q05','q05'):69.81866261434674, ('q06','q06'):195.6373252286935, ('q07','q07'):-17.16956037816925, ('q08','q08'):-14.339120756338502, ('q09','q09'):51.321758487322995, ('q10','q10'):37.16956037816925, ('q11','q11'):94.3391207563385, ('q12','q12'):268.678241512677}

quadratic = {('q01','q02'):28.000000000000007, ('q01','q03'):56.000000000000014, ('q01','q07'):6.933180080632877, ('q01','q08'):13.866360161265755, ('q01','q09'):27.73272032253151, ('q01','q10'):-6.933180080632877, ('q01','q11'):-13.866360161265755, ('q01','q12'):-27.73272032253151, ('q02','q03'):112.00000000000003, ('q02','q07'):13.866360161265755, ('q02','q08'):27.73272032253151, ('q02','q09'):55.46544064506302, ('q02','q10'):-13.866360161265755, ('q02','q11'):-27.73272032253151, ('q02','q12'):-55.46544064506302, ('q03','q07'):27.73272032253151, ('q03','q08'):55.46544064506302, ('q03','q09'):110.93088129012604, ('q03','q10'):-27.73272032253151, ('q03','q11'):-55.46544064506302, ('q03','q12'):-110.93088129012604, ('q04','q05'):28.000000000000007, ('q04','q06'):56.000000000000014, ('q04','q07'):-6.933180080632877, ('q04','q08'):-13.866360161265755, ('q04','q09'):-27.73272032253151, ('q04','q10'):6.933180080632877, ('q04','q11'):13.866360161265755, ('q04','q12'):27.73272032253151, ('q05','q06'):112.00000000000003, ('q05','q07'):-13.866360161265755, ('q05','q08'):-27.73272032253151, ('q05','q09'):-55.46544064506302, ('q05','q10'):13.866360161265755, ('q05','q11'):27.73272032253151, ('q05','q12'):55.46544064506302, ('q06','q07'):-27.73272032253151, ('q06','q08'):-55.46544064506302, ('q06','q09'):-110.93088129012604, ('q06','q10'):27.73272032253151, ('q06','q11'):55.46544064506302, ('q06','q12'):110.93088129012604, ('q07','q08'):40.0, ('q07','q09'):80.0, ('q08','q09'):160.0, ('q10','q11'):40.0, ('q10','q12'):80.0, ('q11','q12'):160.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12     energy num_oc. ...
1    1   0   0   0   0   0   1   0   0   0   0   0 -24.145712      51 ...
69   1   0   0   0   0   0   1   0   0   0   0   0 -24.145712       1 ...
2    0   0   0   0   0   0   1   0   0   0   0   0  -17.16956      38 ...
3    0   1   0   0   0   0   1   0   0   0   0   0 -17.121863      65 ...
70   0   1   0   0   0   0   1   0   0   0   0   0 -17.121863       1 ...
4    1   0   0   0   0   0   0   1   0   0   0   0 -14.382092      54 ...
5    0   0   0   0   0   0   0   1   0   0   0   0 -14.339121      57 ...
6    1   0   0   0   0   0   0   0   0   0   0   0 -13.909331      21 ...
8    1   0   0   0   0   0   0   0   0   0   0   0 -13.909331      42 ...
7    0   1   0   0   0   0   0   0   0   0   0   0 -13.818663      92 ...
33   0   1   0   0   0   0   0   0   0   0   0   0 -13.818663       5 ...
41   0   1   0   0   0   0   0   0   0   0   0   0 -13.818663       2 ...
73   0   1   0   0   0   0   0   0   0

In [23]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  0
Solution x is 3.21699087727594815078E+03
Current  x is 3.21700000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.70000000000000000000E+01
Previous error of x is -9.12272405184921808541E-03
Previous error of y is 1.49814893105570945409E-02


# Matrix Q is
[[   7.02384877   28.           56.            0.            0.
     0.            6.93318008   13.86636016   27.73272032   -6.93318008
   -13.86636016  -27.73272032]
 [   0.           28.04769755  112.            0.            0.
     0.           13.86636016   27.73272032   55.46544065  -13.86636016
   -27.73272032  -55.46544065]
 [   0.            0.          112.09539509    0.            0.
     0.           27.73272032   55.46544065  110.93088129  -27.73272032
   -55.46544065 -110.93088129]
 [   0.            0.            0.            6.97615123   28.
    56.           -6.93318008  -13.86636016  -27.73272032    6.93318008
    13.86636016   27.73272032]
 [   0.            0.        

In [24]:
linear = {('q01','q01'):7.02384877345682, ('q02','q02'):28.047697546913643, ('q03','q03'):112.09539509382729, ('q04','q04'):6.976151226543182, ('q05','q05'):27.952302453086368, ('q06','q06'):111.90460490617275, ('q07','q07'):9.763619702460105, ('q08','q08'):39.52723940492021, ('q09','q09'):159.05447880984042, ('q10','q10'):10.236380297539897, ('q11','q11'):40.47276059507979, ('q12','q12'):160.9455211901596}

quadratic = {('q01','q02'):28.000000000000007, ('q01','q03'):56.000000000000014, ('q01','q07'):6.933180080632877, ('q01','q08'):13.866360161265755, ('q01','q09'):27.73272032253151, ('q01','q10'):-6.933180080632877, ('q01','q11'):-13.866360161265755, ('q01','q12'):-27.73272032253151, ('q02','q03'):112.00000000000003, ('q02','q07'):13.866360161265755, ('q02','q08'):27.73272032253151, ('q02','q09'):55.46544064506302, ('q02','q10'):-13.866360161265755, ('q02','q11'):-27.73272032253151, ('q02','q12'):-55.46544064506302, ('q03','q07'):27.73272032253151, ('q03','q08'):55.46544064506302, ('q03','q09'):110.93088129012604, ('q03','q10'):-27.73272032253151, ('q03','q11'):-55.46544064506302, ('q03','q12'):-110.93088129012604, ('q04','q05'):28.000000000000007, ('q04','q06'):56.000000000000014, ('q04','q07'):-6.933180080632877, ('q04','q08'):-13.866360161265755, ('q04','q09'):-27.73272032253151, ('q04','q10'):6.933180080632877, ('q04','q11'):13.866360161265755, ('q04','q12'):27.73272032253151, ('q05','q06'):112.00000000000003, ('q05','q07'):-13.866360161265755, ('q05','q08'):-27.73272032253151, ('q05','q09'):-55.46544064506302, ('q05','q10'):13.866360161265755, ('q05','q11'):27.73272032253151, ('q05','q12'):55.46544064506302, ('q06','q07'):-27.73272032253151, ('q06','q08'):-55.46544064506302, ('q06','q09'):-110.93088129012604, ('q06','q10'):27.73272032253151, ('q06','q11'):55.46544064506302, ('q06','q12'):110.93088129012604, ('q07','q08'):40.0, ('q07','q09'):80.0, ('q08','q09'):160.0, ('q10','q11'):40.0, ('q10','q12'):80.0, ('q11','q12'):160.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12     energy num_oc. ...
0    0   0   0   0   0   0   0   0   0   0   0   0        0.0     150 ...
56   0   0   0   0   0   0   0   0   0   0   0   0        0.0       1 ...
1    0   0   0   1   0   0   0   0   0   0   0   0   6.976151     178 ...
55   0   0   0   1   0   0   0   0   0   0   0   0   6.976151       1 ...
2    1   0   0   0   0   0   0   0   0   0   0   0   7.023849      66 ...
3    0   0   0   0   0   0   1   0   0   0   0   0    9.76362      45 ...
4    0   0   0   1   0   0   1   0   0   0   0   0   9.806591      92 ...
5    0   0   0   0   0   0   0   0   0   1   0   0   10.23638      52 ...
6    1   0   0   0   0   0   0   0   0   1   0   0  10.327049      35 ...
7    1   0   0   1   0   0   0   0   0   0   0   0       14.0      68 ...
8    0   0   0   0   0   0   1   0   0   1   0   0       20.0      15 ...
9    1   0   0   0   0   0   1   0   0   0   0   0  23.720649      15 ...
10   1   0   0   1   0   0   1   0   0

In [25]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -3
Solution x is 3.21699087727594815078E+03
Current  x is 3.21700000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.70000000000000000000E+01
Previous error of x is -9.12272405184921808541E-03
Previous error of y is 1.49814893105570945409E-02


# Matrix Q is
[[ 0.1123561   0.4375      0.875       0.          0.          0.
   0.10833094  0.21666188  0.43332376 -0.10833094 -0.21666188 -0.43332376]
 [ 0.          0.44346219  1.75        0.          0.          0.
   0.21666188  0.43332376  0.86664751 -0.21666188 -0.43332376 -0.86664751]
 [ 0.          0.          1.76192439  0.          0.          0.
   0.43332376  0.86664751  1.73329502 -0.43332376 -0.86664751 -1.73329502]
 [ 0.          0.          0.          0.1063939   0.4375      0.875
  -0.10833094 -0.21666188 -0.43332376  0.10833094  0.21666188  0.43332376]
 [ 0.          0.          0.          0.          0.43153781  1.75
  -0.21666188 -0.43332376 -0.86664751  0.21666188  0.43332376

In [26]:
linear = {('q01','q01'):0.11235609668210242, ('q02','q02'):0.4434621933642049, ('q03','q03'):1.76192438672841, ('q04','q04'):0.10639390331789762, ('q05','q05'):0.43153780663579533, ('q06','q06'):1.7380756132715907, ('q07','q07'):0.12670246280751282, ('q08','q08'):0.5659049256150257, ('q09','q09'):2.3818098512300514, ('q10','q10'):0.18579753719248718, ('q11','q11'):0.6840950743849744, ('q12','q12'):2.618190148769949}

quadratic = {('q01','q02'):0.4375000000000001, ('q01','q03'):0.8750000000000002, ('q01','q07'):0.10833093875988871, ('q01','q08'):0.21666187751977742, ('q01','q09'):0.43332375503955484, ('q01','q10'):-0.10833093875988871, ('q01','q11'):-0.21666187751977742, ('q01','q12'):-0.43332375503955484, ('q02','q03'):1.7500000000000004, ('q02','q07'):0.21666187751977742, ('q02','q08'):0.43332375503955484, ('q02','q09'):0.8666475100791097, ('q02','q10'):-0.21666187751977742, ('q02','q11'):-0.43332375503955484, ('q02','q12'):-0.8666475100791097, ('q03','q07'):0.43332375503955484, ('q03','q08'):0.8666475100791097, ('q03','q09'):1.7332950201582193, ('q03','q10'):-0.43332375503955484, ('q03','q11'):-0.8666475100791097, ('q03','q12'):-1.7332950201582193, ('q04','q05'):0.4375000000000001, ('q04','q06'):0.8750000000000002, ('q04','q07'):-0.10833093875988871, ('q04','q08'):-0.21666187751977742, ('q04','q09'):-0.43332375503955484, ('q04','q10'):0.10833093875988871, ('q04','q11'):0.21666187751977742, ('q04','q12'):0.43332375503955484, ('q05','q06'):1.7500000000000004, ('q05','q07'):-0.21666187751977742, ('q05','q08'):-0.43332375503955484, ('q05','q09'):-0.8666475100791097, ('q05','q10'):0.21666187751977742, ('q05','q11'):0.43332375503955484, ('q05','q12'):0.8666475100791097, ('q06','q07'):-0.43332375503955484, ('q06','q08'):-0.8666475100791097, ('q06','q09'):-1.7332950201582193, ('q06','q10'):0.43332375503955484, ('q06','q11'):0.8666475100791097, ('q06','q12'):1.7332950201582193, ('q07','q08'):0.625, ('q07','q09'):1.25, ('q08','q09'):2.5, ('q10','q11'):0.625, ('q10','q12'):1.25, ('q11','q12'):2.5}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12   energy num_oc. ...
0    0   0   0   0   0   0   0   0   0   0   0   0      0.0     141 ...
38   0   0   0   0   0   0   0   0   0   0   0   0      0.0       1 ...
50   0   0   0   0   0   0   0   0   0   0   0   0      0.0       1 ...
58   0   0   0   0   0   0   0   0   0   0   0   0      0.0       1 ...
63   0   0   0   0   0   0   0   0   0   0   0   0      0.0       1 ...
1    0   0   0   1   0   0   0   0   0   0   0   0 0.106394      44 ...
61   0   0   0   1   0   0   0   0   0   0   0   0 0.106394       1 ...
2    1   0   0   0   0   0   0   0   0   0   0   0 0.112356     123 ...
47   1   0   0   0   0   0   0   0   0   0   0   0 0.112356       1 ...
3    0   0   0   1   0   0   1   0   0   0   0   0 0.124765      61 ...
4    0   0   0   0   0   0   1   0   0   0   0   0 0.126702      94 ...
5    0   0   0   0   0   0   0   0   0   1   0   0 0.185798      39 ...
6    1   0   0   0   0   0   0   0   0   1   0   0 0.189823     

In [27]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -6
Solution x is 3.21699087727594815078E+03
Current  x is 3.21700000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.70000000000000000000E+01
Previous error of x is -9.12272405184921808541E-03
Previous error of y is 1.49814893105570945409E-02


# Matrix Q is
[[ 0.00208162  0.00683594  0.01367188  0.          0.          0.
   0.00169267  0.00338534  0.00677068 -0.00169267 -0.00338534 -0.00677068]
 [ 0.          0.00758121  0.02734375  0.          0.          0.
   0.00338534  0.00677068  0.01354137 -0.00338534 -0.00677068 -0.01354137]
 [ 0.          0.          0.0288343   0.          0.          0.
   0.00677068  0.01354137  0.02708273 -0.00677068 -0.01354137 -0.02708273]
 [ 0.          0.          0.          0.00133635  0.00683594  0.01367188
  -0.00169267 -0.00338534 -0.00677068  0.00169267  0.00338534  0.00677068]
 [ 0.          0.          0.          0.          0.00609066  0.02734375
  -0.00338534 -0.00677068 -0.01354137  0.00338534 

In [28]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -9
Solution x is 3.21699087727594815078E+03
Current  x is 3.21700000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.70000000000000000000E+01
Previous error of x is -9.12272405184921808541E-03
Previous error of y is 1.49814893105570945409E-02


# Matrix Q is
[[ 7.32825165e-05  1.06811523e-04  2.13623047e-04  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.64479831e-05  5.28959662e-05
   1.05791932e-04 -2.64479831e-05 -5.28959662e-05 -1.05791932e-04]
 [ 0.00000000e+00  1.99970795e-04  4.27246094e-04  0.00000000e+00
   0.00000000e+00  0.00000000e+00  5.28959662e-05  1.05791932e-04
   2.11583865e-04 -5.28959662e-05 -1.05791932e-04 -2.11583865e-04]
 [ 0.00000000e+00  0.00000000e+00  6.13564636e-04  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.05791932e-04  2.11583865e-04
   4.23167730e-04 -1.05791932e-04 -2.11583865e-04 -4.23167730e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.98767548e-05
   1.06811523e-04  2.13623047e-04

In [29]:
linear = {('q01','q01'):7.328251651722491e-05, ('q02','q02'):0.00019997079475319983, ('q03','q03'):0.0006135646363813997, ('q04','q04'):-1.9876754798474913e-05, ('q05','q05'):1.3652252121800173e-05, ('q06','q06'):0.0002409275511186004, ('q07','q07'):-0.000423533295976362, ('q08','q08'):-0.000770772646640224, ('q09','q09'):-0.001236369512030448, ('q10','q10'):0.0004998272412888619, ('q11','q11'):0.0010759484278902239, ('q12','q12'):0.0024570726370304477}

quadratic = {('q01','q02'):0.00010681152343750003, ('q01','q03'):0.00021362304687500005, ('q01','q07'):2.6447983095675954e-05, ('q01','q08'):5.289596619135191e-05, ('q01','q09'):0.00010579193238270382, ('q01','q10'):-2.6447983095675954e-05, ('q01','q11'):-5.289596619135191e-05, ('q01','q12'):-0.00010579193238270382, ('q02','q03'):0.0004272460937500001, ('q02','q07'):5.289596619135191e-05, ('q02','q08'):0.00010579193238270382, ('q02','q09'):0.00021158386476540764, ('q02','q10'):-5.289596619135191e-05, ('q02','q11'):-0.00010579193238270382, ('q02','q12'):-0.00021158386476540764, ('q03','q07'):0.00010579193238270382, ('q03','q08'):0.00021158386476540764, ('q03','q09'):0.00042316772953081527, ('q03','q10'):-0.00010579193238270382, ('q03','q11'):-0.00021158386476540764, ('q03','q12'):-0.00042316772953081527, ('q04','q05'):0.00010681152343750003, ('q04','q06'):0.00021362304687500005, ('q04','q07'):-2.6447983095675954e-05, ('q04','q08'):-5.289596619135191e-05, ('q04','q09'):-0.00010579193238270382, ('q04','q10'):2.6447983095675954e-05, ('q04','q11'):5.289596619135191e-05, ('q04','q12'):0.00010579193238270382, ('q05','q06'):0.0004272460937500001, ('q05','q07'):-5.289596619135191e-05, ('q05','q08'):-0.00010579193238270382, ('q05','q09'):-0.00021158386476540764, ('q05','q10'):5.289596619135191e-05, ('q05','q11'):0.00010579193238270382, ('q05','q12'):0.00021158386476540764, ('q06','q07'):-0.00010579193238270382, ('q06','q08'):-0.00021158386476540764, ('q06','q09'):-0.00042316772953081527, ('q06','q10'):0.00010579193238270382, ('q06','q11'):0.00021158386476540764, ('q06','q12'):0.00042316772953081527, ('q07','q08'):0.000152587890625, ('q07','q09'):0.00030517578125, ('q08','q09'):0.0006103515625, ('q10','q11'):0.000152587890625, ('q10','q12'):0.00030517578125, ('q11','q12'):0.0006103515625}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12    energy num_oc. ...
0    0   0   0   0   0   1   1   1   1   0   0   0 -0.001862     228 ...
26   0   0   0   0   0   1   1   1   1   0   0   0 -0.001862       1 ...
1    0   0   0   1   0   1   1   1   1   0   0   0 -0.001854     442 ...
22   0   0   0   1   0   1   1   1   1   0   0   0 -0.001854       1 ...
25   0   0   0   1   0   1   1   1   1   0   0   0 -0.001854       1 ...
2    0   0   0   1   1   0   1   1   1   0   0   0 -0.001817      85 ...
3    0   0   0   0   1   1   1   1   1   0   0   0 -0.001792      67 ...
4    0   0   0   0   0   1   0   1   1   0   0   0 -0.001791      49 ...
5    0   0   0   1   1   0   0   1   1   0   0   0 -0.001772      41 ...
6    0   0   0   1   0   1   0   1   1   0   0   0 -0.001756      50 ...
24   0   0   0   1   0   1   0   1   1   0   0   0 -0.001756       1 ...
7    0   0   0   0   1   0   1   1   1   0   0   0 -0.001719       2 ...
8    0   0   0   0   1   0   0   1   1   0   0   0 

In [30]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -9
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699218750000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69863281250000000000E+01
Previous error of x is -1.31022405184921808541E-03
Previous error of y is 1.30961431055709454085E-03


# Matrix Q is
[[ 4.47953513e-05  1.06811523e-04  2.13623047e-04  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.64479831e-05  5.28959662e-05
   1.05791932e-04 -2.64479831e-05 -5.28959662e-05 -1.05791932e-04]
 [ 0.00000000e+00  1.42996464e-04  4.27246094e-04  0.00000000e+00
   0.00000000e+00  0.00000000e+00  5.28959662e-05  1.05791932e-04
   2.11583865e-04 -5.28959662e-05 -1.05791932e-04 -2.11583865e-04]
 [ 0.00000000e+00  0.00000000e+00  4.99615976e-04  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.05791932e-04  2.11583865e-04
   4.23167730e-04 -1.05791932e-04 -2.11583865e-04 -4.23167730e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  8.61041041e-06
   1.06811523e-04  2.13623047e-04

In [31]:
linear = {('q01','q01'):4.479535131058611e-05, ('q02','q02'):0.00014299646433992222, ('q03','q03'):0.0004996159755548445, ('q04','q04'):8.610410408163899e-06, ('q05','q05'):7.06265825350778e-05, ('q06','q06'):0.00035487621194515566, ('q07','q07'):4.732388824154588e-06, ('q08','q08'):8.575872296080917e-05, ('q09','q09'):0.00047669322717161836, ('q10','q10'):7.156155648834541e-05, ('q11','q11'):0.00021941705828919082, ('q12','q12'):0.0007440098978283816}

quadratic = {('q01','q02'):0.00010681152343750003, ('q01','q03'):0.00021362304687500005, ('q01','q07'):2.6447983095675954e-05, ('q01','q08'):5.289596619135191e-05, ('q01','q09'):0.00010579193238270382, ('q01','q10'):-2.6447983095675954e-05, ('q01','q11'):-5.289596619135191e-05, ('q01','q12'):-0.00010579193238270382, ('q02','q03'):0.0004272460937500001, ('q02','q07'):5.289596619135191e-05, ('q02','q08'):0.00010579193238270382, ('q02','q09'):0.00021158386476540764, ('q02','q10'):-5.289596619135191e-05, ('q02','q11'):-0.00010579193238270382, ('q02','q12'):-0.00021158386476540764, ('q03','q07'):0.00010579193238270382, ('q03','q08'):0.00021158386476540764, ('q03','q09'):0.00042316772953081527, ('q03','q10'):-0.00010579193238270382, ('q03','q11'):-0.00021158386476540764, ('q03','q12'):-0.00042316772953081527, ('q04','q05'):0.00010681152343750003, ('q04','q06'):0.00021362304687500005, ('q04','q07'):-2.6447983095675954e-05, ('q04','q08'):-5.289596619135191e-05, ('q04','q09'):-0.00010579193238270382, ('q04','q10'):2.6447983095675954e-05, ('q04','q11'):5.289596619135191e-05, ('q04','q12'):0.00010579193238270382, ('q05','q06'):0.0004272460937500001, ('q05','q07'):-5.289596619135191e-05, ('q05','q08'):-0.00010579193238270382, ('q05','q09'):-0.00021158386476540764, ('q05','q10'):5.289596619135191e-05, ('q05','q11'):0.00010579193238270382, ('q05','q12'):0.00021158386476540764, ('q06','q07'):-0.00010579193238270382, ('q06','q08'):-0.00021158386476540764, ('q06','q09'):-0.00042316772953081527, ('q06','q10'):0.00010579193238270382, ('q06','q11'):0.00021158386476540764, ('q06','q12'):0.00042316772953081527, ('q07','q08'):0.000152587890625, ('q07','q09'):0.00030517578125, ('q08','q09'):0.0006103515625, ('q10','q11'):0.000152587890625, ('q10','q12'):0.00030517578125, ('q11','q12'):0.0006103515625}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12    energy num_oc. ...
0    0   0   0   1   0   0   1   0   0   0   0   0 -0.000013      86 ...
1    0   0   0   0   0   0   0   0   0   0   0   0       0.0      73 ...
75   0   0   0   0   0   0   0   0   0   0   0   0       0.0       1 ...
2    0   0   0   0   0   0   1   0   0   0   0   0  0.000005      51 ...
3    0   0   0   1   0   0   0   0   0   0   0   0  0.000009      57 ...
4    0   0   0   0   1   0   1   0   0   0   0   0  0.000022      89 ...
5    0   0   0   1   0   0   0   1   0   0   0   0  0.000041      80 ...
6    1   0   0   0   0   0   0   0   0   0   0   0  0.000045      17 ...
7    0   0   0   0   1   0   0   1   0   0   0   0  0.000051     130 ...
74   0   0   0   0   1   0   0   1   0   0   0   0  0.000051       1 ...
8    1   0   0   1   0   0   0   0   0   0   0   0  0.000053      18 ...
9    1   0   0   1   0   0   1   0   0   0   0   0  0.000058      17 ...
10   0   0   0   0   1   0   0   0   0   0   0   0 

In [32]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -9
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699023437500000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69843750000000000000E+01
Previous error of x is 6.42900948150781914592E-04
Previous error of y is -6.43510689442905459146E-04


# Matrix Q is
[[ 1.78375727e-05  1.06811523e-04  2.13623047e-04  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.64479831e-05  5.28959662e-05
   1.05791932e-04 -2.64479831e-05 -5.28959662e-05 -1.05791932e-04]
 [ 0.00000000e+00  8.90809071e-05  4.27246094e-04  0.00000000e+00
   0.00000000e+00  0.00000000e+00  5.28959662e-05  1.05791932e-04
   2.11583865e-04 -5.28959662e-05 -1.05791932e-04 -2.11583865e-04]
 [ 0.00000000e+00  0.00000000e+00  3.91784861e-04  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.05791932e-04  2.11583865e-04
   4.23167730e-04 -1.05791932e-04 -2.11583865e-04 -4.23167730e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.55681890e-05
   1.06811523e-04  2.13623047e-04

In [33]:
linear = {('q01','q01'):1.7837572696601058e-05, ('q02','q02'):8.908090711195212e-05, ('q03','q03'):0.00039178486109890426, ('q04','q04'):3.5568189022148955e-05, ('q05','q05'):0.00012454213976304792, ('q06','q06'):0.00046270732640109585, ('q07','q07'):5.4578351054934e-05, ('q08','q08'):0.00018545064742236798, ('q09','q09'):0.000676077076094736, ('q10','q10'):2.1715594257566005e-05, ('q11','q11'):0.00011972513382763202, ('q12','q12'):0.000544626048905264}

quadratic = {('q01','q02'):0.00010681152343750003, ('q01','q03'):0.00021362304687500005, ('q01','q07'):2.6447983095675954e-05, ('q01','q08'):5.289596619135191e-05, ('q01','q09'):0.00010579193238270382, ('q01','q10'):-2.6447983095675954e-05, ('q01','q11'):-5.289596619135191e-05, ('q01','q12'):-0.00010579193238270382, ('q02','q03'):0.0004272460937500001, ('q02','q07'):5.289596619135191e-05, ('q02','q08'):0.00010579193238270382, ('q02','q09'):0.00021158386476540764, ('q02','q10'):-5.289596619135191e-05, ('q02','q11'):-0.00010579193238270382, ('q02','q12'):-0.00021158386476540764, ('q03','q07'):0.00010579193238270382, ('q03','q08'):0.00021158386476540764, ('q03','q09'):0.00042316772953081527, ('q03','q10'):-0.00010579193238270382, ('q03','q11'):-0.00021158386476540764, ('q03','q12'):-0.00042316772953081527, ('q04','q05'):0.00010681152343750003, ('q04','q06'):0.00021362304687500005, ('q04','q07'):-2.6447983095675954e-05, ('q04','q08'):-5.289596619135191e-05, ('q04','q09'):-0.00010579193238270382, ('q04','q10'):2.6447983095675954e-05, ('q04','q11'):5.289596619135191e-05, ('q04','q12'):0.00010579193238270382, ('q05','q06'):0.0004272460937500001, ('q05','q07'):-5.289596619135191e-05, ('q05','q08'):-0.00010579193238270382, ('q05','q09'):-0.00021158386476540764, ('q05','q10'):5.289596619135191e-05, ('q05','q11'):0.00010579193238270382, ('q05','q12'):0.00021158386476540764, ('q06','q07'):-0.00010579193238270382, ('q06','q08'):-0.00021158386476540764, ('q06','q09'):-0.00042316772953081527, ('q06','q10'):0.00010579193238270382, ('q06','q11'):0.00021158386476540764, ('q06','q12'):0.00042316772953081527, ('q07','q08'):0.000152587890625, ('q07','q09'):0.00030517578125, ('q08','q09'):0.0006103515625, ('q10','q11'):0.000152587890625, ('q10','q12'):0.00030517578125, ('q11','q12'):0.0006103515625}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12   energy num_oc. ...
0    0   0   0   0   0   0   0   0   0   0   0   0      0.0      89 ...
1    1   0   0   0   0   0   0   0   0   1   0   0 0.000013      98 ...
2    1   0   0   0   0   0   0   0   0   0   0   0 0.000018      73 ...
3    0   0   0   0   0   0   0   0   0   1   0   0 0.000022      63 ...
4    0   0   0   1   0   0   0   0   0   0   0   0 0.000036      57 ...
71   0   0   0   1   0   0   0   0   0   0   0   0 0.000036       1 ...
5    1   0   0   1   0   0   0   0   0   0   0   0 0.000053      37 ...
6    0   0   0   0   0   0   1   0   0   0   0   0 0.000055      38 ...
7    0   1   0   0   0   0   0   0   0   1   0   0 0.000058      91 ...
8    0   0   0   1   0   0   1   0   0   0   0   0 0.000064      49 ...
9    1   0   0   1   0   0   0   0   0   1   0   0 0.000075      19 ...
10   0   0   0   0   0   0   1   0   0   1   0   0 0.000076      21 ...
11   0   0   0   1   0   0   0   0   0   1   0   0 0.000084     

In [34]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -12
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699023437500000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69843750000000000000E+01
Previous error of x is 6.42900948150781914592E-04
Previous error of y is -6.43510689442905459146E-04


# Matrix Q is
[[-6.90931007e-07  1.66893005e-06  3.33786011e-06  0.00000000e+00
   0.00000000e+00  0.00000000e+00  4.13249736e-07  8.26499472e-07
   1.65299894e-06 -4.13249736e-07 -8.26499472e-07 -1.65299894e-06]
 [ 0.00000000e+00 -5.47396987e-07  6.67572021e-06  0.00000000e+00
   0.00000000e+00  0.00000000e+00  8.26499472e-07  1.65299894e-06
   3.30599789e-06 -8.26499472e-07 -1.65299894e-06 -3.30599789e-06]
 [ 0.00000000e+00  0.00000000e+00  2.24306613e-06  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.65299894e-06  3.30599789e-06
   6.61199577e-06 -1.65299894e-06 -3.30599789e-06 -6.61199577e-06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.52539603e-06
   1.66893005e-06  3.33786011e-0

In [35]:
linear = {('q01','q01'):-6.909310069190092e-07, ('q02','q02'):-5.473969869825494e-07, ('q03','q03'):2.243066133456777e-06, ('q04','q04'):1.525396033774478e-06, ('q05','q05'):3.885257094404425e-06, ('q06','q06'):1.1108374296230724e-05, ('q07','q07'):2.6499687475894056e-06, ('q08','q08'):6.492030390686624e-06, ('q09','q09'):1.7752432363404497e-05, ('q10','q10'):-1.4578758520815931e-06, ('q11','q11'):-1.7236588086553733e-06, ('q12','q12'):1.3210539647205036e-06}

quadratic = {('q01','q02'):1.668930053710938e-06, ('q01','q03'):3.337860107421876e-06, ('q01','q07'):4.132497358699368e-07, ('q01','q08'):8.264994717398736e-07, ('q01','q09'):1.6529989434797471e-06, ('q01','q10'):-4.132497358699368e-07, ('q01','q11'):-8.264994717398736e-07, ('q01','q12'):-1.6529989434797471e-06, ('q02','q03'):6.675720214843752e-06, ('q02','q07'):8.264994717398736e-07, ('q02','q08'):1.6529989434797471e-06, ('q02','q09'):3.3059978869594943e-06, ('q02','q10'):-8.264994717398736e-07, ('q02','q11'):-1.6529989434797471e-06, ('q02','q12'):-3.3059978869594943e-06, ('q03','q07'):1.6529989434797471e-06, ('q03','q08'):3.3059978869594943e-06, ('q03','q09'):6.611995773918989e-06, ('q03','q10'):-1.6529989434797471e-06, ('q03','q11'):-3.3059978869594943e-06, ('q03','q12'):-6.611995773918989e-06, ('q04','q05'):1.668930053710938e-06, ('q04','q06'):3.337860107421876e-06, ('q04','q07'):-4.132497358699368e-07, ('q04','q08'):-8.264994717398736e-07, ('q04','q09'):-1.6529989434797471e-06, ('q04','q10'):4.132497358699368e-07, ('q04','q11'):8.264994717398736e-07, ('q04','q12'):1.6529989434797471e-06, ('q05','q06'):6.675720214843752e-06, ('q05','q07'):-8.264994717398736e-07, ('q05','q08'):-1.6529989434797471e-06, ('q05','q09'):-3.3059978869594943e-06, ('q05','q10'):8.264994717398736e-07, ('q05','q11'):1.6529989434797471e-06, ('q05','q12'):3.3059978869594943e-06, ('q06','q07'):-1.6529989434797471e-06, ('q06','q08'):-3.3059978869594943e-06, ('q06','q09'):-6.611995773918989e-06, ('q06','q10'):1.6529989434797471e-06, ('q06','q11'):3.3059978869594943e-06, ('q06','q12'):6.611995773918989e-06, ('q07','q08'):2.384185791015625e-06, ('q07','q09'):4.76837158203125e-06, ('q08','q09'):9.5367431640625e-06, ('q10','q11'):2.384185791015625e-06, ('q10','q12'):4.76837158203125e-06, ('q11','q12'):9.5367431640625e-06}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12    energy num_oc. ...
0    1   1   0   0   0   0   0   0   0   1   1   0 -0.000004     312 ...
18   1   1   0   0   0   0   0   0   0   1   1   0 -0.000004       1 ...
1    0   1   0   0   0   0   0   0   0   0   1   0 -0.000004      89 ...
2    0   1   0   0   0   0   0   0   0   1   1   0 -0.000004     127 ...
3    1   1   0   0   0   0   0   0   0   0   1   0 -0.000004     147 ...
4    0   0   1   0   0   0   0   0   0   1   1   0 -0.000004      35 ...
5    1   0   0   0   0   0   0   0   0   0   1   0 -0.000003      17 ...
6    1   1   0   0   0   0   0   0   0   0   0   1 -0.000003      66 ...
7    0   0   1   0   0   0   0   0   0   0   0   1 -0.000003      33 ...
8    0   1   0   0   0   0   0   0   0   1   0   0 -0.000003      24 ...
9    0   0   1   0   0   0   0   0   0   0   1   0 -0.000003       4 ...
10   1   0   0   0   0   0   0   0   0   1   1   0 -0.000003       3 ...
11   1   0   0   0   0   0   0   0   0   1   0   0 

In [36]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -12
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699096679687500000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69851074218750000000E+01
Previous error of x is -8.95209268492180854082E-05
Previous error of y is 8.89111855570945408545E-05


# Matrix Q is
[[ 5.72714865e-07  1.66893005e-06  3.33786011e-06  0.00000000e+00
   0.00000000e+00  0.00000000e+00  4.13249736e-07  8.26499472e-07
   1.65299894e-06 -4.13249736e-07 -8.26499472e-07 -1.65299894e-06]
 [ 0.00000000e+00  1.97989476e-06  6.67572021e-06  0.00000000e+00
   0.00000000e+00  0.00000000e+00  8.26499472e-07  1.65299894e-06
   3.30599789e-06 -8.26499472e-07 -1.65299894e-06 -3.30599789e-06]
 [ 0.00000000e+00  0.00000000e+00  7.29764962e-06  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.65299894e-06  3.30599789e-06
   6.61199577e-06 -1.65299894e-06 -3.30599789e-06 -6.61199577e-06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.61750162e-07
   1.66893005e-06  3.33786011e-0

In [37]:
linear = {('q01','q01'):5.727148650948904e-07, ('q02','q02'):1.9798947570452496e-06, ('q03','q03'):7.297649621512375e-06, ('q04','q04'):2.617501617605785e-07, ('q05','q05'):1.3579653503766258e-06, ('q06','q06'):6.0537908081751275e-06, ('q07','q07'):3.1343926835549276e-07, ('q08','q08'):1.8189714322187977e-06, ('q09','q09'):8.406314446468845e-06, ('q10','q10'):8.786536271523199e-07, ('q11','q11'):2.949400149812452e-06, ('q12','q12'):1.0667171881656155e-05}

quadratic = {('q01','q02'):1.668930053710938e-06, ('q01','q03'):3.337860107421876e-06, ('q01','q07'):4.132497358699368e-07, ('q01','q08'):8.264994717398736e-07, ('q01','q09'):1.6529989434797471e-06, ('q01','q10'):-4.132497358699368e-07, ('q01','q11'):-8.264994717398736e-07, ('q01','q12'):-1.6529989434797471e-06, ('q02','q03'):6.675720214843752e-06, ('q02','q07'):8.264994717398736e-07, ('q02','q08'):1.6529989434797471e-06, ('q02','q09'):3.3059978869594943e-06, ('q02','q10'):-8.264994717398736e-07, ('q02','q11'):-1.6529989434797471e-06, ('q02','q12'):-3.3059978869594943e-06, ('q03','q07'):1.6529989434797471e-06, ('q03','q08'):3.3059978869594943e-06, ('q03','q09'):6.611995773918989e-06, ('q03','q10'):-1.6529989434797471e-06, ('q03','q11'):-3.3059978869594943e-06, ('q03','q12'):-6.611995773918989e-06, ('q04','q05'):1.668930053710938e-06, ('q04','q06'):3.337860107421876e-06, ('q04','q07'):-4.132497358699368e-07, ('q04','q08'):-8.264994717398736e-07, ('q04','q09'):-1.6529989434797471e-06, ('q04','q10'):4.132497358699368e-07, ('q04','q11'):8.264994717398736e-07, ('q04','q12'):1.6529989434797471e-06, ('q05','q06'):6.675720214843752e-06, ('q05','q07'):-8.264994717398736e-07, ('q05','q08'):-1.6529989434797471e-06, ('q05','q09'):-3.3059978869594943e-06, ('q05','q10'):8.264994717398736e-07, ('q05','q11'):1.6529989434797471e-06, ('q05','q12'):3.3059978869594943e-06, ('q06','q07'):-1.6529989434797471e-06, ('q06','q08'):-3.3059978869594943e-06, ('q06','q09'):-6.611995773918989e-06, ('q06','q10'):1.6529989434797471e-06, ('q06','q11'):3.3059978869594943e-06, ('q06','q12'):6.611995773918989e-06, ('q07','q08'):2.384185791015625e-06, ('q07','q09'):4.76837158203125e-06, ('q08','q09'):9.5367431640625e-06, ('q10','q11'):2.384185791015625e-06, ('q10','q12'):4.76837158203125e-06, ('q11','q12'):9.5367431640625e-06}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12   energy num_oc. ...
0    0   0   0   0   0   0   0   0   0   0   0   0      0.0      53 ...
2    0   0   0   0   0   0   0   0   0   0   0   0      0.0      13 ...
3    0   0   0   1   0   0   1   0   0   0   0   0      0.0      69 ...
5    0   0   0   1   0   0   0   0   0   0   0   0      0.0      22 ...
1    0   0   0   0   0   0   1   0   0   0   0   0      0.0      97 ...
6    0   0   0   0   0   0   1   0   0   0   0   0      0.0      22 ...
4    1   0   0   0   0   0   0   0   0   0   0   0 0.000001      84 ...
8    1   0   0   0   0   0   0   0   0   0   0   0 0.000001      16 ...
10   1   0   0   1   0   0   0   0   0   0   0   0 0.000001      28 ...
12   0   0   0   0   1   0   1   0   0   0   0   0 0.000001      16 ...
7    0   0   0   0   0   0   0   0   0   1   0   0 0.000001      12 ...
13   0   0   0   0   0   0   0   0   0   1   0   0 0.000001       1 ...
9    1   0   0   0   0   0   0   0   0   1   0   0 0.000001     

In [38]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -15
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699096679687500000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69851074218750000000E+01
Previous error of x is -8.95209268492180854082E-05
Previous error of y is 8.89111855570945408545E-05


# Matrix Q is
[[ 2.59545520e-08  2.60770321e-08  5.21540642e-08  0.00000000e+00
   0.00000000e+00  0.00000000e+00  6.45702712e-09  1.29140542e-08
   2.58281085e-08 -6.45702712e-09 -1.29140542e-08 -2.58281085e-08]
 [ 0.00000000e+00  6.49476200e-08  1.04308128e-07  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.29140542e-08  2.58281085e-08
   5.16562170e-08 -1.29140542e-08 -2.58281085e-08 -5.16562170e-08]
 [ 0.00000000e+00  0.00000000e+00  1.82049304e-07  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.58281085e-08  5.16562170e-08
   1.03312434e-07 -2.58281085e-08 -5.16562170e-08 -1.03312434e-07]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.29160359e-08
   2.60770321e-08  5.21540642e-0

In [39]:
linear = {('q01','q01'):2.595455198070285e-08, ('q02','q02'):6.49476200060224e-08, ('q03','q03'):1.820493041905116e-07, ('q04','q04'):-1.2916035936086147e-08, ('q05','q05'):-1.2793555827555592e-08, ('q06','q06'):2.656695252335562e-08, ('q07','q07'):-2.601267167864691e-08, ('q08','q08'):-3.3398891864984244e-08, ('q09','q09'):7.708022239269796e-09, ('q10','q10'):4.463912317095648e-08, ('q11','q11'):1.0790469783422253e-07, ('q12','q12'):2.903152016376834e-07}

quadratic = {('q01','q02'):2.6077032089233405e-08, ('q01','q03'):5.215406417846681e-08, ('q01','q07'):6.457027122967762e-09, ('q01','q08'):1.2914054245935525e-08, ('q01','q09'):2.582810849187105e-08, ('q01','q10'):-6.457027122967762e-09, ('q01','q11'):-1.2914054245935525e-08, ('q01','q12'):-2.582810849187105e-08, ('q02','q03'):1.0430812835693362e-07, ('q02','q07'):1.2914054245935525e-08, ('q02','q08'):2.582810849187105e-08, ('q02','q09'):5.16562169837421e-08, ('q02','q10'):-1.2914054245935525e-08, ('q02','q11'):-2.582810849187105e-08, ('q02','q12'):-5.16562169837421e-08, ('q03','q07'):2.582810849187105e-08, ('q03','q08'):5.16562169837421e-08, ('q03','q09'):1.033124339674842e-07, ('q03','q10'):-2.582810849187105e-08, ('q03','q11'):-5.16562169837421e-08, ('q03','q12'):-1.033124339674842e-07, ('q04','q05'):2.6077032089233405e-08, ('q04','q06'):5.215406417846681e-08, ('q04','q07'):-6.457027122967762e-09, ('q04','q08'):-1.2914054245935525e-08, ('q04','q09'):-2.582810849187105e-08, ('q04','q10'):6.457027122967762e-09, ('q04','q11'):1.2914054245935525e-08, ('q04','q12'):2.582810849187105e-08, ('q05','q06'):1.0430812835693362e-07, ('q05','q07'):-1.2914054245935525e-08, ('q05','q08'):-2.582810849187105e-08, ('q05','q09'):-5.16562169837421e-08, ('q05','q10'):1.2914054245935525e-08, ('q05','q11'):2.582810849187105e-08, ('q05','q12'):5.16562169837421e-08, ('q06','q07'):-2.582810849187105e-08, ('q06','q08'):-5.16562169837421e-08, ('q06','q09'):-1.033124339674842e-07, ('q06','q10'):2.582810849187105e-08, ('q06','q11'):5.16562169837421e-08, ('q06','q12'):1.033124339674842e-07, ('q07','q08'):3.725290298461914e-08, ('q07','q09'):7.450580596923828e-08, ('q08','q09'):1.4901161193847656e-07, ('q10','q11'):3.725290298461914e-08, ('q10','q12'):7.450580596923828e-08, ('q11','q12'):1.4901161193847656e-07}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12 energy num_oc. chain_b.
0    0   0   0   1   1   0   1   1   0   0   0   0   -0.0     279      0.0
32   0   0   0   1   1   0   1   1   0   0   0   0   -0.0       1 0.083333
47   0   0   0   1   1   0   1   1   0   0   0   0   -0.0       1 0.083333
1    0   0   0   0   1   0   1   1   0   0   0   0   -0.0      77      0.0
38   0   0   0   0   1   0   1   1   0   0   0   0   -0.0       1 0.083333
2    0   0   0   0   0   1   1   1   0   0   0   0   -0.0      44      0.0
3    0   0   0   0   1   0   0   1   0   0   0   0   -0.0     104      0.0
4    0   0   0   1   1   0   0   1   0   0   0   0   -0.0     187      0.0
44   0   0   0   1   1   0   0   1   0   0   0   0   -0.0       1 0.083333
5    0   0   0   1   1   0   0   0   1   0   0   0   -0.0     104      0.0
6    0   0   0   0   0   1   0   0   1   0   0   0   -0.0      34      0.0
7    0   0   0   1   0   0   0   1   0   0   0   0   -0.0      29      0.0
8    0   0   0   0   0   

In [40]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -15
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087524414062500E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850158691406250000E+01
Previous error of x is 2.03180752578191459179E-06
Previous error of y is -2.64154881790545914555E-06


# Matrix Q is
[[ 6.21008530e-09  2.60770321e-08  5.21540642e-08  0.00000000e+00
   0.00000000e+00  0.00000000e+00  6.45702712e-09  1.29140542e-08
   2.58281085e-08 -6.45702712e-09 -1.29140542e-08 -2.58281085e-08]
 [ 0.00000000e+00  2.54586866e-08  1.04308128e-07  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.29140542e-08  2.58281085e-08
   5.16562170e-08 -1.29140542e-08 -2.58281085e-08 -5.16562170e-08]
 [ 0.00000000e+00  0.00000000e+00  1.03071437e-07  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.58281085e-08  5.16562170e-08
   1.03312434e-07 -2.58281085e-08 -5.16562170e-08 -1.03312434e-07]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.82843074e-09
   2.60770321e-08  5.21540642e-0

In [41]:
linear = {('q01','q01'):6.210085300769655e-09, ('q02','q02'):2.5458686646156012e-08, ('q03','q03'):1.0307143747077883e-07, ('q04','q04'):6.828430743847047e-09, ('q05','q05'):2.6695377532310794e-08, ('q06','q06'):1.055448192430884e-07, ('q07','q07'):1.0495601423026085e-08, ('q08','q08'):3.9617654338361743e-08, ('q09','q09'):1.5374111464596177e-07, ('q10','q10'):8.130850069283485e-09, ('q11','q11'):3.488815163087654e-08, ('q12','q12'):1.4428210923099136e-07}

quadratic = {('q01','q02'):2.6077032089233405e-08, ('q01','q03'):5.215406417846681e-08, ('q01','q07'):6.457027122967762e-09, ('q01','q08'):1.2914054245935525e-08, ('q01','q09'):2.582810849187105e-08, ('q01','q10'):-6.457027122967762e-09, ('q01','q11'):-1.2914054245935525e-08, ('q01','q12'):-2.582810849187105e-08, ('q02','q03'):1.0430812835693362e-07, ('q02','q07'):1.2914054245935525e-08, ('q02','q08'):2.582810849187105e-08, ('q02','q09'):5.16562169837421e-08, ('q02','q10'):-1.2914054245935525e-08, ('q02','q11'):-2.582810849187105e-08, ('q02','q12'):-5.16562169837421e-08, ('q03','q07'):2.582810849187105e-08, ('q03','q08'):5.16562169837421e-08, ('q03','q09'):1.033124339674842e-07, ('q03','q10'):-2.582810849187105e-08, ('q03','q11'):-5.16562169837421e-08, ('q03','q12'):-1.033124339674842e-07, ('q04','q05'):2.6077032089233405e-08, ('q04','q06'):5.215406417846681e-08, ('q04','q07'):-6.457027122967762e-09, ('q04','q08'):-1.2914054245935525e-08, ('q04','q09'):-2.582810849187105e-08, ('q04','q10'):6.457027122967762e-09, ('q04','q11'):1.2914054245935525e-08, ('q04','q12'):2.582810849187105e-08, ('q05','q06'):1.0430812835693362e-07, ('q05','q07'):-1.2914054245935525e-08, ('q05','q08'):-2.582810849187105e-08, ('q05','q09'):-5.16562169837421e-08, ('q05','q10'):1.2914054245935525e-08, ('q05','q11'):2.582810849187105e-08, ('q05','q12'):5.16562169837421e-08, ('q06','q07'):-2.582810849187105e-08, ('q06','q08'):-5.16562169837421e-08, ('q06','q09'):-1.033124339674842e-07, ('q06','q10'):2.582810849187105e-08, ('q06','q11'):5.16562169837421e-08, ('q06','q12'):1.033124339674842e-07, ('q07','q08'):3.725290298461914e-08, ('q07','q09'):7.450580596923828e-08, ('q08','q09'):1.4901161193847656e-07, ('q10','q11'):3.725290298461914e-08, ('q10','q12'):7.450580596923828e-08, ('q11','q12'):1.4901161193847656e-07}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12 energy num_oc. chain_b.
0    0   0   0   0   0   0   0   0   0   0   0   0    0.0     121      0.0
35   0   0   0   0   0   0   0   0   0   0   0   0    0.0       1 0.083333
63   0   0   0   0   0   0   0   0   0   0   0   0    0.0       1 0.083333
1    1   0   0   0   0   0   0   0   0   0   0   0    0.0      88      0.0
2    0   0   0   1   0   0   0   0   0   0   0   0    0.0      19      0.0
3    1   0   0   0   0   0   0   0   0   1   0   0    0.0     143      0.0
4    0   0   0   0   0   0   0   0   0   1   0   0    0.0     101      0.0
47   0   0   0   0   0   0   0   0   0   1   0   0    0.0       1 0.083333
5    0   0   0   0   0   0   1   0   0   0   0   0    0.0      32      0.0
6    0   0   0   1   0   0   1   0   0   0   0   0    0.0      12      0.0
7    1   0   0   1   0   0   0   0   0   0   0   0    0.0      22      0.0
8    0   0   0   0   0   0   1   0   0   1   0   0    0.0      20      0.0
9    0   1   0   0   0   

In [42]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -18
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087524414062500E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850158691406250000E+01
Previous error of x is 2.03180752578191459179E-06
Previous error of y is -2.64154881790545914555E-06


# Matrix Q is
[[ 6.32168164e-11  4.07453626e-10  8.14907253e-10  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00891049e-10  2.01782098e-10
   4.03564195e-10 -1.00891049e-10 -2.01782098e-10 -4.03564195e-10]
 [ 0.00000000e+00  3.30160446e-10  1.62981451e-09  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.01782098e-10  4.03564195e-10
   8.07128390e-10 -2.01782098e-10 -4.03564195e-10 -8.07128390e-10]
 [ 0.00000000e+00  0.00000000e+00  1.47522814e-09  0.00000000e+00
   0.00000000e+00  0.00000000e+00  4.03564195e-10  8.07128390e-10
   1.61425678e-09 -4.03564195e-10 -8.07128390e-10 -1.61425678e-09]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.40509997e-10
   4.07453626e-10  8.14907253e-1

In [43]:
linear = {('q01','q01'):6.321681640623105e-11, ('q02','q02'):3.3016044600959807e-10, ('q03','q03'):1.47522814480774e-09, ('q04','q04'):1.405099967909049e-10, ('q05','q05'):4.847468067789458e-10, ('q06','q06'):1.7844008663464354e-09, ('q07','q07'):2.9331611189258104e-10, ('q08','q08'):8.776705283524991e-10, ('q09','q09'):2.9194942749743466e-09, ('q10','q10'):-2.277807325244016e-12, ('q11','q11'):2.86482689916849e-10, ('q12','q12'):1.7371185981030462e-09}

quadratic = {('q01','q02'):4.0745362639427195e-10, ('q01','q03'):8.149072527885439e-10, ('q01','q07'):1.0089104879637129e-10, ('q01','q08'):2.0178209759274257e-10, ('q01','q09'):4.0356419518548514e-10, ('q01','q10'):-1.0089104879637129e-10, ('q01','q11'):-2.0178209759274257e-10, ('q01','q12'):-4.0356419518548514e-10, ('q02','q03'):1.6298145055770878e-09, ('q02','q07'):2.0178209759274257e-10, ('q02','q08'):4.0356419518548514e-10, ('q02','q09'):8.071283903709703e-10, ('q02','q10'):-2.0178209759274257e-10, ('q02','q11'):-4.0356419518548514e-10, ('q02','q12'):-8.071283903709703e-10, ('q03','q07'):4.0356419518548514e-10, ('q03','q08'):8.071283903709703e-10, ('q03','q09'):1.6142567807419406e-09, ('q03','q10'):-4.0356419518548514e-10, ('q03','q11'):-8.071283903709703e-10, ('q03','q12'):-1.6142567807419406e-09, ('q04','q05'):4.0745362639427195e-10, ('q04','q06'):8.149072527885439e-10, ('q04','q07'):-1.0089104879637129e-10, ('q04','q08'):-2.0178209759274257e-10, ('q04','q09'):-4.0356419518548514e-10, ('q04','q10'):1.0089104879637129e-10, ('q04','q11'):2.0178209759274257e-10, ('q04','q12'):4.0356419518548514e-10, ('q05','q06'):1.6298145055770878e-09, ('q05','q07'):-2.0178209759274257e-10, ('q05','q08'):-4.0356419518548514e-10, ('q05','q09'):-8.071283903709703e-10, ('q05','q10'):2.0178209759274257e-10, ('q05','q11'):4.0356419518548514e-10, ('q05','q12'):8.071283903709703e-10, ('q06','q07'):-4.0356419518548514e-10, ('q06','q08'):-8.071283903709703e-10, ('q06','q09'):-1.6142567807419406e-09, ('q06','q10'):4.0356419518548514e-10, ('q06','q11'):8.071283903709703e-10, ('q06','q12'):1.6142567807419406e-09, ('q07','q08'):5.820766091346741e-10, ('q07','q09'):1.1641532182693481e-09, ('q08','q09'):2.3283064365386963e-09, ('q10','q11'):5.820766091346741e-10, ('q10','q12'):1.1641532182693481e-09, ('q11','q12'):2.3283064365386963e-09}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12 energy num_oc. chain_b.
0    1   0   0   0   0   0   0   0   0   1   0   0   -0.0     124      0.0
1    0   0   0   0   0   0   0   0   0   1   0   0   -0.0     194      0.0
2    0   0   0   0   0   0   0   0   0   0   0   0    0.0      88      0.0
3    1   0   0   0   0   0   0   0   0   0   0   0    0.0      37      0.0
4    0   1   0   0   0   0   0   0   0   1   0   0    0.0      93      0.0
5    0   0   0   1   0   0   0   0   0   0   0   0    0.0      30      0.0
6    1   0   0   0   0   0   0   0   0   0   1   0    0.0      33      0.0
7    1   0   0   1   0   0   0   0   0   1   0   0    0.0      20      0.0
8    1   0   0   1   0   0   0   0   0   0   0   0    0.0      12      0.0
9    0   1   0   0   0   0   0   0   0   0   1   0    0.0      82      0.0
58   0   1   0   0   0   0   0   0   0   0   1   0    0.0       1 0.083333
10   0   0   0   1   0   0   0   0   0   1   0   0    0.0      23      0.0
11   0   0   0   0   0   

In [44]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -18
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087905883789062E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850196838378906250E+01
Previous error of x is -1.78288973984308540821E-06
Previous error of y is 1.17314844771954085445E-06


# Matrix Q is
[[ 1.66052576e-10  4.07453626e-10  8.14907253e-10  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00891049e-10  2.01782098e-10
   4.03564195e-10 -1.00891049e-10 -2.01782098e-10 -4.03564195e-10]
 [ 0.00000000e+00  5.35831964e-10  1.62981451e-09  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.01782098e-10  4.03564195e-10
   8.07128390e-10 -2.01782098e-10 -4.03564195e-10 -8.07128390e-10]
 [ 0.00000000e+00  0.00000000e+00  1.88657118e-09  0.00000000e+00
   0.00000000e+00  0.00000000e+00  4.03564195e-10  8.07128390e-10
   1.61425678e-09 -4.03564195e-10 -8.07128390e-10 -1.61425678e-09]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.76742376e-11
   4.07453626e-10  8.14907253e-1

In [45]:
linear = {('q01','q01'):1.6605257561817332e-10, ('q02','q02'):5.358319644334826e-10, ('q03','q03'):1.8865711816555093e-09, ('q04','q04'):3.767423757896265e-11, ('q05','q05'):2.790752883550613e-10, ('q06','q06'):1.3730578294986664e-09, ('q07','q07'):1.0316887909142801e-10, ('q08','q08'):4.973760627501931e-10, ('q09','q09'):2.158905343769734e-09, ('q10','q10'):1.8786942547590905e-10, ('q11','q11'):6.667771555191552e-10, ('q12','q12'):2.4977075293076585e-09}

quadratic = {('q01','q02'):4.0745362639427195e-10, ('q01','q03'):8.149072527885439e-10, ('q01','q07'):1.0089104879637129e-10, ('q01','q08'):2.0178209759274257e-10, ('q01','q09'):4.0356419518548514e-10, ('q01','q10'):-1.0089104879637129e-10, ('q01','q11'):-2.0178209759274257e-10, ('q01','q12'):-4.0356419518548514e-10, ('q02','q03'):1.6298145055770878e-09, ('q02','q07'):2.0178209759274257e-10, ('q02','q08'):4.0356419518548514e-10, ('q02','q09'):8.071283903709703e-10, ('q02','q10'):-2.0178209759274257e-10, ('q02','q11'):-4.0356419518548514e-10, ('q02','q12'):-8.071283903709703e-10, ('q03','q07'):4.0356419518548514e-10, ('q03','q08'):8.071283903709703e-10, ('q03','q09'):1.6142567807419406e-09, ('q03','q10'):-4.0356419518548514e-10, ('q03','q11'):-8.071283903709703e-10, ('q03','q12'):-1.6142567807419406e-09, ('q04','q05'):4.0745362639427195e-10, ('q04','q06'):8.149072527885439e-10, ('q04','q07'):-1.0089104879637129e-10, ('q04','q08'):-2.0178209759274257e-10, ('q04','q09'):-4.0356419518548514e-10, ('q04','q10'):1.0089104879637129e-10, ('q04','q11'):2.0178209759274257e-10, ('q04','q12'):4.0356419518548514e-10, ('q05','q06'):1.6298145055770878e-09, ('q05','q07'):-2.0178209759274257e-10, ('q05','q08'):-4.0356419518548514e-10, ('q05','q09'):-8.071283903709703e-10, ('q05','q10'):2.0178209759274257e-10, ('q05','q11'):4.0356419518548514e-10, ('q05','q12'):8.071283903709703e-10, ('q06','q07'):-4.0356419518548514e-10, ('q06','q08'):-8.071283903709703e-10, ('q06','q09'):-1.6142567807419406e-09, ('q06','q10'):4.0356419518548514e-10, ('q06','q11'):8.071283903709703e-10, ('q06','q12'):1.6142567807419406e-09, ('q07','q08'):5.820766091346741e-10, ('q07','q09'):1.1641532182693481e-09, ('q08','q09'):2.3283064365386963e-09, ('q10','q11'):5.820766091346741e-10, ('q10','q12'):1.1641532182693481e-09, ('q11','q12'):2.3283064365386963e-09}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12 energy num_oc. chain_b.
0    0   0   0   0   0   0   0   0   0   0   0   0    0.0      70      0.0
1    0   0   0   1   0   0   0   0   0   0   0   0    0.0      36      0.0
2    0   0   0   1   0   0   1   0   0   0   0   0    0.0      85      0.0
3    0   0   0   0   0   0   1   0   0   0   0   0    0.0      71      0.0
4    1   0   0   0   0   0   0   0   0   0   0   0    0.0      27      0.0
72   1   0   0   0   0   0   0   0   0   0   0   0    0.0       1 0.083333
5    0   0   0   0   1   0   1   0   0   0   0   0    0.0      71      0.0
6    0   0   0   0   0   0   0   0   0   1   0   0    0.0      58      0.0
7    1   0   0   1   0   0   0   0   0   0   0   0    0.0      29      0.0
8    1   0   0   0   0   0   0   0   0   1   0   0    0.0      69      0.0
9    0   0   0   0   1   0   0   0   0   0   0   0    0.0      22      0.0
10   0   0   0   0   0   0   1   0   0   1   0   0    0.0      54      0.0
73   0   0   0   0   0   

In [46]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -21
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087905883789062E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850196838378906250E+01
Previous error of x is -1.78288973984308540821E-06
Previous error of y is 1.17314844771954085445E-06


# Matrix Q is
[[ 9.61526186e-12  6.36646291e-12  1.27329258e-11  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.57642264e-12  3.15284527e-12
   6.30569055e-12 -1.57642264e-12 -3.15284527e-12 -6.30569055e-12]
 [ 0.00000000e+00  2.24137552e-11  2.54658516e-11  0.00000000e+00
   0.00000000e+00  0.00000000e+00  3.15284527e-12  6.30569055e-12
   1.26113811e-11 -3.15284527e-12 -6.30569055e-12 -1.26113811e-11]
 [ 0.00000000e+00  0.00000000e+00  5.75604362e-11  0.00000000e+00
   0.00000000e+00  0.00000000e+00  6.30569055e-12  1.26113811e-11
   2.52227622e-11 -6.30569055e-12 -1.26113811e-11 -2.52227622e-11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -6.43203040e-12
   6.36646291e-12  1.27329258e-1

In [47]:
linear = {('q01','q01'):9.615261855553291e-12, ('q02','q02'):2.2413755167311832e-11, ('q03','q03'):5.7560436159444665e-11, ('q04','q04'):-6.4320303993480426e-12, ('q05','q05'):-9.680829342490835e-12, ('q06','q06'):-6.6287328601606724e-12, ('q07','q07'):-3.0200473945977448e-12, ('q08','q08'):-1.4926212803308475e-12, ('q09','q09'):1.520465147479687e-11, ('q10','q10'):7.567520903462386e-12, ('q11','q11'):1.9682515315789412e-11, ('q12','q12'):5.7554924667037396e-11}

quadratic = {('q01','q02'):6.366462912410499e-12, ('q01','q03'):1.2732925824820999e-11, ('q01','q07'):1.5764226374433013e-12, ('q01','q08'):3.1528452748866027e-12, ('q01','q09'):6.305690549773205e-12, ('q01','q10'):-1.5764226374433013e-12, ('q01','q11'):-3.1528452748866027e-12, ('q01','q12'):-6.305690549773205e-12, ('q02','q03'):2.5465851649641997e-11, ('q02','q07'):3.1528452748866027e-12, ('q02','q08'):6.305690549773205e-12, ('q02','q09'):1.261138109954641e-11, ('q02','q10'):-3.1528452748866027e-12, ('q02','q11'):-6.305690549773205e-12, ('q02','q12'):-1.261138109954641e-11, ('q03','q07'):6.305690549773205e-12, ('q03','q08'):1.261138109954641e-11, ('q03','q09'):2.522276219909282e-11, ('q03','q10'):-6.305690549773205e-12, ('q03','q11'):-1.261138109954641e-11, ('q03','q12'):-2.522276219909282e-11, ('q04','q05'):6.366462912410499e-12, ('q04','q06'):1.2732925824820999e-11, ('q04','q07'):-1.5764226374433013e-12, ('q04','q08'):-3.1528452748866027e-12, ('q04','q09'):-6.305690549773205e-12, ('q04','q10'):1.5764226374433013e-12, ('q04','q11'):3.1528452748866027e-12, ('q04','q12'):6.305690549773205e-12, ('q05','q06'):2.5465851649641997e-11, ('q05','q07'):-3.1528452748866027e-12, ('q05','q08'):-6.305690549773205e-12, ('q05','q09'):-1.261138109954641e-11, ('q05','q10'):3.1528452748866027e-12, ('q05','q11'):6.305690549773205e-12, ('q05','q12'):1.261138109954641e-11, ('q06','q07'):-6.305690549773205e-12, ('q06','q08'):-1.261138109954641e-11, ('q06','q09'):-2.522276219909282e-11, ('q06','q10'):6.305690549773205e-12, ('q06','q11'):1.261138109954641e-11, ('q06','q12'):2.522276219909282e-11, ('q07','q08'):9.094947017729282e-12, ('q07','q09'):1.8189894035458565e-11, ('q08','q09'):3.637978807091713e-11, ('q10','q11'):9.094947017729282e-12, ('q10','q12'):1.8189894035458565e-11, ('q11','q12'):3.637978807091713e-11}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12 energy num_oc. chain_b.
0    0   0   0   0   0   1   1   1   0   0   0   0   -0.0     132      0.0
1    0   0   0   0   0   1   0   1   0   0   0   0   -0.0      90      0.0
2    0   0   0   1   1   0   0   1   0   0   0   0   -0.0     192      0.0
3    0   0   0   1   0   1   1   1   0   0   0   0   -0.0      46      0.0
4    0   0   0   1   1   0   1   1   0   0   0   0   -0.0      90      0.0
5    0   0   0   1   0   1   0   1   0   0   0   0   -0.0      15      0.0
6    0   0   0   1   1   0   1   0   0   0   0   0   -0.0      33      0.0
7    0   0   0   0   1   0   0   1   0   0   0   0   -0.0      97      0.0
17   0   0   0   0   1   0   0   1   0   0   0   0   -0.0       1 0.083333
8    0   0   0   1   0   1   0   0   1   0   0   0   -0.0      57      0.0
9    0   0   0   0   0   1   0   0   1   0   0   0   -0.0      75      0.0
10   0   0   0   0   0   1   1   0   0   0   0   0   -0.0       6      0.0
11   0   0   0   0   1   

In [48]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -21
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087715148925781E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850182533264160156E+01
Previous error of x is 1.24458892969414591789E-07
Previous error of y is -2.57363026889834145550E-07


# Matrix Q is
[[ 1.61160428e-12  6.36646291e-12  1.27329258e-11  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.57642264e-12  3.15284527e-12
   6.30569055e-12 -1.57642264e-12 -3.15284527e-12 -6.30569055e-12]
 [ 0.00000000e+00  6.40644003e-12  2.54658516e-11  0.00000000e+00
   0.00000000e+00  0.00000000e+00  3.15284527e-12  6.30569055e-12
   1.26113811e-11 -3.15284527e-12 -6.30569055e-12 -1.26113811e-11]
 [ 0.00000000e+00  0.00000000e+00  2.55458059e-11  0.00000000e+00
   0.00000000e+00  0.00000000e+00  6.30569055e-12  1.26113811e-11
   2.52227622e-11 -6.30569055e-12 -1.26113811e-11 -2.52227622e-11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.57162717e-12
   6.36646291e-12  1.27329258e-1

In [49]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -21
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087524414062500E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850168228149414062E+01
Previous error of x is 2.03180752578191459179E-06
Previous error of y is -1.68787450149920914555E-06


# Matrix Q is
[[-6.39205257e-12  6.36646291e-12  1.27329258e-11  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.57642264e-12  3.15284527e-12
   6.30569055e-12 -1.57642264e-12 -3.15284527e-12 -6.30569055e-12]
 [ 0.00000000e+00 -9.60087369e-12  2.54658516e-11  0.00000000e+00
   0.00000000e+00  0.00000000e+00  3.15284527e-12  6.30569055e-12
   1.26113811e-11 -3.15284527e-12 -6.30569055e-12 -1.26113811e-11]
 [ 0.00000000e+00  0.00000000e+00 -6.46882156e-12  0.00000000e+00
   0.00000000e+00  0.00000000e+00  6.30569055e-12  1.26113811e-11
   2.52227622e-11 -6.30569055e-12 -1.26113811e-11 -2.52227622e-11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  9.57528403e-12
   6.36646291e-12  1.27329258e-1

In [50]:
linear = {('q01','q01'):-6.392052573755081e-12, ('q02','q02'):-9.600873691304913e-12, ('q03','q03'):-6.468821557788829e-12, ('q04','q04'):9.57528402996033e-12, ('q05','q05'):2.233379951612591e-11, ('q06','q06'):5.740052485707282e-11, ('q07','q07'):1.1653410298834794e-11, ('q08','q08'):2.785429410653423e-11, ('q09','q09'):7.389848224852702e-11, ('q10','q10'):-7.105936789970153e-12, ('q11','q11'):-9.664400071075664e-12, ('q12','q12'):-1.1389061066927597e-12}

quadratic = {('q01','q02'):6.366462912410499e-12, ('q01','q03'):1.2732925824820999e-11, ('q01','q07'):1.5764226374433013e-12, ('q01','q08'):3.1528452748866027e-12, ('q01','q09'):6.305690549773205e-12, ('q01','q10'):-1.5764226374433013e-12, ('q01','q11'):-3.1528452748866027e-12, ('q01','q12'):-6.305690549773205e-12, ('q02','q03'):2.5465851649641997e-11, ('q02','q07'):3.1528452748866027e-12, ('q02','q08'):6.305690549773205e-12, ('q02','q09'):1.261138109954641e-11, ('q02','q10'):-3.1528452748866027e-12, ('q02','q11'):-6.305690549773205e-12, ('q02','q12'):-1.261138109954641e-11, ('q03','q07'):6.305690549773205e-12, ('q03','q08'):1.261138109954641e-11, ('q03','q09'):2.522276219909282e-11, ('q03','q10'):-6.305690549773205e-12, ('q03','q11'):-1.261138109954641e-11, ('q03','q12'):-2.522276219909282e-11, ('q04','q05'):6.366462912410499e-12, ('q04','q06'):1.2732925824820999e-11, ('q04','q07'):-1.5764226374433013e-12, ('q04','q08'):-3.1528452748866027e-12, ('q04','q09'):-6.305690549773205e-12, ('q04','q10'):1.5764226374433013e-12, ('q04','q11'):3.1528452748866027e-12, ('q04','q12'):6.305690549773205e-12, ('q05','q06'):2.5465851649641997e-11, ('q05','q07'):-3.1528452748866027e-12, ('q05','q08'):-6.305690549773205e-12, ('q05','q09'):-1.261138109954641e-11, ('q05','q10'):3.1528452748866027e-12, ('q05','q11'):6.305690549773205e-12, ('q05','q12'):1.261138109954641e-11, ('q06','q07'):-6.305690549773205e-12, ('q06','q08'):-1.261138109954641e-11, ('q06','q09'):-2.522276219909282e-11, ('q06','q10'):6.305690549773205e-12, ('q06','q11'):1.261138109954641e-11, ('q06','q12'):2.522276219909282e-11, ('q07','q08'):9.094947017729282e-12, ('q07','q09'):1.8189894035458565e-11, ('q08','q09'):3.637978807091713e-11, ('q10','q11'):9.094947017729282e-12, ('q10','q12'):1.8189894035458565e-11, ('q11','q12'):3.637978807091713e-11}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12 energy num_oc. chain_b.
0    0   0   1   0   0   0   0   0   0   1   1   0   -0.0     230      0.0
1    0   0   1   0   0   0   0   0   0   0   0   1   -0.0     157      0.0
2    1   0   1   0   0   0   0   0   0   0   0   1   -0.0      51      0.0
3    1   1   0   0   0   0   0   0   0   1   1   0   -0.0      23      0.0
4    1   0   1   0   0   0   0   0   0   1   1   0   -0.0      49      0.0
5    1   1   0   0   0   0   0   0   0   0   0   1   -0.0      17      0.0
6    1   0   1   0   0   0   0   0   0   1   0   1   -0.0      34      0.0
17   1   0   1   0   0   0   0   0   0   1   0   1   -0.0       1 0.083333
7    0   1   1   0   0   0   0   0   0   0   0   1   -0.0      89      0.0
8    1   1   0   0   0   0   0   0   0   0   1   0   -0.0      23      0.0
9    0   0   1   0   0   0   0   0   0   0   1   0   -0.0      57      0.0
10   0   0   1   0   0   0   0   0   0   1   0   1   -0.0      63      0.0
22   0   0   1   0   0   

In [51]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -21
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087715148925781E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850182533264160156E+01
Previous error of x is 1.24458892969414591789E-07
Previous error of y is -2.57363026889834145550E-07


# Matrix Q is
[[ 1.61160428e-12  6.36646291e-12  1.27329258e-11  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.57642264e-12  3.15284527e-12
   6.30569055e-12 -1.57642264e-12 -3.15284527e-12 -6.30569055e-12]
 [ 0.00000000e+00  6.40644003e-12  2.54658516e-11  0.00000000e+00
   0.00000000e+00  0.00000000e+00  3.15284527e-12  6.30569055e-12
   1.26113811e-11 -3.15284527e-12 -6.30569055e-12 -1.26113811e-11]
 [ 0.00000000e+00  0.00000000e+00  2.55458059e-11  0.00000000e+00
   0.00000000e+00  0.00000000e+00  6.30569055e-12  1.26113811e-11
   2.52227622e-11 -6.30569055e-12 -1.26113811e-11 -2.52227622e-11]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.57162717e-12
   6.36646291e-12  1.27329258e-1

In [52]:
linear = {('q01','q01'):1.6116042844765682e-12, ('q02','q02'):6.406440025158385e-12, ('q03','q03'):2.554580587513777e-11, ('q04','q04'):1.5716271717286816e-12, ('q05','q05'):6.326485799662612e-12, ('q06','q06'):2.5385897424146222e-11, ('q07','q07'):4.316679553549497e-12, ('q08','q08'):1.3180832615963637e-11, ('q09','q09'):4.455155926738584e-11, ('q10','q10'):2.3079395531514407e-13, ('q11','q11'):5.00906141949493e-12, ('q12','q12'):2.8208016874448427e-11}

quadratic = {('q01','q02'):6.366462912410499e-12, ('q01','q03'):1.2732925824820999e-11, ('q01','q07'):1.5764226374433013e-12, ('q01','q08'):3.1528452748866027e-12, ('q01','q09'):6.305690549773205e-12, ('q01','q10'):-1.5764226374433013e-12, ('q01','q11'):-3.1528452748866027e-12, ('q01','q12'):-6.305690549773205e-12, ('q02','q03'):2.5465851649641997e-11, ('q02','q07'):3.1528452748866027e-12, ('q02','q08'):6.305690549773205e-12, ('q02','q09'):1.261138109954641e-11, ('q02','q10'):-3.1528452748866027e-12, ('q02','q11'):-6.305690549773205e-12, ('q02','q12'):-1.261138109954641e-11, ('q03','q07'):6.305690549773205e-12, ('q03','q08'):1.261138109954641e-11, ('q03','q09'):2.522276219909282e-11, ('q03','q10'):-6.305690549773205e-12, ('q03','q11'):-1.261138109954641e-11, ('q03','q12'):-2.522276219909282e-11, ('q04','q05'):6.366462912410499e-12, ('q04','q06'):1.2732925824820999e-11, ('q04','q07'):-1.5764226374433013e-12, ('q04','q08'):-3.1528452748866027e-12, ('q04','q09'):-6.305690549773205e-12, ('q04','q10'):1.5764226374433013e-12, ('q04','q11'):3.1528452748866027e-12, ('q04','q12'):6.305690549773205e-12, ('q05','q06'):2.5465851649641997e-11, ('q05','q07'):-3.1528452748866027e-12, ('q05','q08'):-6.305690549773205e-12, ('q05','q09'):-1.261138109954641e-11, ('q05','q10'):3.1528452748866027e-12, ('q05','q11'):6.305690549773205e-12, ('q05','q12'):1.261138109954641e-11, ('q06','q07'):-6.305690549773205e-12, ('q06','q08'):-1.261138109954641e-11, ('q06','q09'):-2.522276219909282e-11, ('q06','q10'):6.305690549773205e-12, ('q06','q11'):1.261138109954641e-11, ('q06','q12'):2.522276219909282e-11, ('q07','q08'):9.094947017729282e-12, ('q07','q09'):1.8189894035458565e-11, ('q08','q09'):3.637978807091713e-11, ('q10','q11'):9.094947017729282e-12, ('q10','q12'):1.8189894035458565e-11, ('q11','q12'):3.637978807091713e-11}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12 energy num_oc. chain_b.
0    0   0   0   0   0   0   0   0   0   0   0   0    0.0      76      0.0
1    0   0   0   0   0   0   0   0   0   1   0   0    0.0     145      0.0
2    1   0   0   0   0   0   0   0   0   1   0   0    0.0     147      0.0
57   1   0   0   0   0   0   0   0   0   1   0   0    0.0       1 0.083333
3    0   0   0   1   0   0   0   0   0   0   0   0    0.0      15      0.0
4    1   0   0   0   0   0   0   0   0   0   0   0    0.0      33      0.0
5    1   0   0   1   0   0   0   0   0   0   0   0    0.0       8      0.0
6    0   0   0   1   0   0   0   0   0   1   0   0    0.0      33      0.0
7    1   0   0   1   0   0   0   0   0   1   0   0    0.0      34      0.0
8    1   0   0   0   0   0   0   0   0   0   1   0    0.0      82      0.0
9    0   1   0   0   0   0   0   0   0   1   0   0    0.0      70      0.0
10   0   0   0   1   0   0   1   0   0   0   0   0    0.0       7      0.0
11   0   0   0   0   0   

In [53]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -24
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087715148925781E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850182533264160156E+01
Previous error of x is 1.24458892969414591789E-07
Previous error of y is -2.57363026889834145550E-07


# Matrix Q is
[[ 2.73675653e-14  9.94759830e-14  1.98951966e-13  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.46316037e-14  4.92632074e-14
   9.85264148e-14 -2.46316037e-14 -4.92632074e-14 -9.85264148e-14]
 [ 0.00000000e+00  1.04473122e-13  3.97903932e-13  0.00000000e+00
   0.00000000e+00  0.00000000e+00  4.92632074e-14  9.85264148e-14
   1.97052830e-13 -4.92632074e-14 -9.85264148e-14 -1.97052830e-13]
 [ 0.00000000e+00  0.00000000e+00  4.07898210e-13  0.00000000e+00
   0.00000000e+00  0.00000000e+00  9.85264148e-14  1.97052830e-13
   3.94105659e-13 -9.85264148e-14 -1.97052830e-13 -3.94105659e-13]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.23704262e-14
   9.94759830e-14  1.98951966e-1

In [54]:
linear = {('q01','q01'):2.7367565298346428e-14, ('q02','q02'):1.0447312209989988e-13, ('q03','q03'):4.0789821021262784e-13, ('q04','q04'):2.2370426204860585e-14, ('q05','q05'):9.447884391292818e-14, ('q06','q06'):3.879096538386845e-13, ('q07','q07'):2.9089498667765205e-13, ('q08','q08'):6.528442469313141e-13, ('q09','q09'):1.5899055881666683e-12, ('q10','q10'):-2.1984071310164206e-13, ('q11','q11'):-3.686271526272741e-13, ('q12','q12'):-4.5303721095050806e-13}

quadratic = {('q01','q02'):9.947598300641405e-14, ('q01','q03'):1.989519660128281e-13, ('q01','q07'):2.4631603710051583e-14, ('q01','q08'):4.926320742010317e-14, ('q01','q09'):9.852641484020633e-14, ('q01','q10'):-2.4631603710051583e-14, ('q01','q11'):-4.926320742010317e-14, ('q01','q12'):-9.852641484020633e-14, ('q02','q03'):3.979039320256562e-13, ('q02','q07'):4.926320742010317e-14, ('q02','q08'):9.852641484020633e-14, ('q02','q09'):1.9705282968041267e-13, ('q02','q10'):-4.926320742010317e-14, ('q02','q11'):-9.852641484020633e-14, ('q02','q12'):-1.9705282968041267e-13, ('q03','q07'):9.852641484020633e-14, ('q03','q08'):1.9705282968041267e-13, ('q03','q09'):3.9410565936082534e-13, ('q03','q10'):-9.852641484020633e-14, ('q03','q11'):-1.9705282968041267e-13, ('q03','q12'):-3.9410565936082534e-13, ('q04','q05'):9.947598300641405e-14, ('q04','q06'):1.989519660128281e-13, ('q04','q07'):-2.4631603710051583e-14, ('q04','q08'):-4.926320742010317e-14, ('q04','q09'):-9.852641484020633e-14, ('q04','q10'):2.4631603710051583e-14, ('q04','q11'):4.926320742010317e-14, ('q04','q12'):9.852641484020633e-14, ('q05','q06'):3.979039320256562e-13, ('q05','q07'):-4.926320742010317e-14, ('q05','q08'):-9.852641484020633e-14, ('q05','q09'):-1.9705282968041267e-13, ('q05','q10'):4.926320742010317e-14, ('q05','q11'):9.852641484020633e-14, ('q05','q12'):1.9705282968041267e-13, ('q06','q07'):-9.852641484020633e-14, ('q06','q08'):-1.9705282968041267e-13, ('q06','q09'):-3.9410565936082534e-13, ('q06','q10'):9.852641484020633e-14, ('q06','q11'):1.9705282968041267e-13, ('q06','q12'):3.9410565936082534e-13, ('q07','q08'):1.4210854715202004e-13, ('q07','q09'):2.8421709430404007e-13, ('q08','q09'):5.684341886080801e-13, ('q10','q11'):1.4210854715202004e-13, ('q10','q12'):2.8421709430404007e-13, ('q11','q12'):5.684341886080801e-13}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12 energy num_oc. chain_b.
0    0   1   0   0   0   0   0   0   0   0   0   1   -0.0     156      0.0
1    0   1   0   0   0   0   0   0   0   1   0   1   -0.0      83      0.0
2    1   1   0   0   0   0   0   0   0   1   0   1   -0.0      92      0.0
3    1   0   0   0   0   0   0   0   0   0   0   1   -0.0     114      0.0
4    1   1   0   0   0   0   0   0   0   0   0   1   -0.0      67      0.0
5    1   0   0   0   0   0   0   0   0   1   1   0   -0.0      86      0.0
6    0   1   0   0   0   0   0   0   0   1   1   0   -0.0      71      0.0
18   0   1   0   0   0   0   0   0   0   1   1   0   -0.0       3 0.083333
46   0   1   0   0   0   0   0   0   0   1   1   0   -0.0       1 0.083333
7    1   0   0   0   0   0   0   0   0   1   0   1   -0.0      34      0.0
8    0   0   1   0   0   0   0   0   0   1   0   1   -0.0      87      0.0
9    1   1   0   0   0   0   0   0   0   0   1   1   -0.0      17      0.0
10   0   0   0   0   0   

In [55]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -24
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727069854736E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850184917449951172E+01
Previous error of x is 5.24960341863334178925E-09
Previous error of y is -1.89444477882716455497E-08


# Matrix Q is
[[ 2.83170356e-14  9.94759830e-14  1.98951966e-13  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.46316037e-14  4.92632074e-14
   9.85264148e-14 -2.46316037e-14 -4.92632074e-14 -9.85264148e-14]
 [ 0.00000000e+00  1.06372063e-13  3.97903932e-13  0.00000000e+00
   0.00000000e+00  0.00000000e+00  4.92632074e-14  9.85264148e-14
   1.97052830e-13 -4.92632074e-14 -9.85264148e-14 -1.97052830e-13]
 [ 0.00000000e+00  0.00000000e+00  4.11696092e-13  0.00000000e+00
   0.00000000e+00  0.00000000e+00  9.85264148e-14  1.97052830e-13
   3.94105659e-13 -9.85264148e-14 -1.97052830e-13 -3.94105659e-13]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.14209559e-14
   9.94759830e-14  1.98951966e-1

In [56]:
linear = {('q01','q01'):2.8317035634891185e-14, ('q02','q02'):1.0637206277298939e-13, ('q03','q03'):4.1169609155880686e-13, ('q04','q04'):2.1420955868315837e-14, ('q05','q05'):9.25799032398387e-14, ('q06','q06'):3.8411177249250545e-13, ('q07','q07'):5.594117859464131e-14, ('q08','q08'):1.8293663076529266e-13, ('q09','q09'):6.500903558346254e-13, ('q10','q10'):1.5113094981368708e-14, ('q11','q11'):1.0128046353874744e-13, ('q12','q12'):4.86778021381535e-13}

quadratic = {('q01','q02'):9.947598300641405e-14, ('q01','q03'):1.989519660128281e-13, ('q01','q07'):2.4631603710051583e-14, ('q01','q08'):4.926320742010317e-14, ('q01','q09'):9.852641484020633e-14, ('q01','q10'):-2.4631603710051583e-14, ('q01','q11'):-4.926320742010317e-14, ('q01','q12'):-9.852641484020633e-14, ('q02','q03'):3.979039320256562e-13, ('q02','q07'):4.926320742010317e-14, ('q02','q08'):9.852641484020633e-14, ('q02','q09'):1.9705282968041267e-13, ('q02','q10'):-4.926320742010317e-14, ('q02','q11'):-9.852641484020633e-14, ('q02','q12'):-1.9705282968041267e-13, ('q03','q07'):9.852641484020633e-14, ('q03','q08'):1.9705282968041267e-13, ('q03','q09'):3.9410565936082534e-13, ('q03','q10'):-9.852641484020633e-14, ('q03','q11'):-1.9705282968041267e-13, ('q03','q12'):-3.9410565936082534e-13, ('q04','q05'):9.947598300641405e-14, ('q04','q06'):1.989519660128281e-13, ('q04','q07'):-2.4631603710051583e-14, ('q04','q08'):-4.926320742010317e-14, ('q04','q09'):-9.852641484020633e-14, ('q04','q10'):2.4631603710051583e-14, ('q04','q11'):4.926320742010317e-14, ('q04','q12'):9.852641484020633e-14, ('q05','q06'):3.979039320256562e-13, ('q05','q07'):-4.926320742010317e-14, ('q05','q08'):-9.852641484020633e-14, ('q05','q09'):-1.9705282968041267e-13, ('q05','q10'):4.926320742010317e-14, ('q05','q11'):9.852641484020633e-14, ('q05','q12'):1.9705282968041267e-13, ('q06','q07'):-9.852641484020633e-14, ('q06','q08'):-1.9705282968041267e-13, ('q06','q09'):-3.9410565936082534e-13, ('q06','q10'):9.852641484020633e-14, ('q06','q11'):1.9705282968041267e-13, ('q06','q12'):3.9410565936082534e-13, ('q07','q08'):1.4210854715202004e-13, ('q07','q09'):2.8421709430404007e-13, ('q08','q09'):5.684341886080801e-13, ('q10','q11'):1.4210854715202004e-13, ('q10','q12'):2.8421709430404007e-13, ('q11','q12'):5.684341886080801e-13}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12 energy num_oc. chain_b.
0    0   0   0   0   0   0   0   0   0   0   0   0    0.0      88      0.0
1    0   0   0   0   0   0   0   0   0   1   0   0    0.0      51      0.0
78   0   0   0   0   0   0   0   0   0   1   0   0    0.0       1 0.083333
79   0   0   0   0   0   0   0   0   0   1   0   0    0.0       1 0.083333
2    1   0   0   0   0   0   0   0   0   1   0   0    0.0      61      0.0
74   1   0   0   0   0   0   0   0   0   1   0   0    0.0       1 0.083333
3    0   0   0   1   0   0   0   0   0   0   0   0    0.0      71      0.0
4    1   0   0   0   0   0   0   0   0   0   0   0    0.0      68      0.0
73   1   0   0   0   0   0   0   0   0   0   0   0    0.0       1 0.083333
5    1   0   0   1   0   0   0   0   0   0   0   0    0.0      41      0.0
6    0   0   0   1   0   0   1   0   0   0   0   0    0.0      66      0.0
7    0   0   0   0   0   0   1   0   0   0   0   0    0.0      54      0.0
8    0   0   0   1   0   

In [57]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -27
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727069854736E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850184917449951172E+01
Previous error of x is 5.24960341863334178925E-09
Previous error of y is -1.89444477882716455497E-08


# Matrix Q is
[[ 8.19583044e-16  1.55431223e-15  3.10862447e-15  0.00000000e+00
   0.00000000e+00  0.00000000e+00  3.84868808e-16  7.69737616e-16
   1.53947523e-15 -3.84868808e-16 -7.69737616e-16 -1.53947523e-15]
 [ 0.00000000e+00  2.41632221e-15  6.21724894e-15  0.00000000e+00
   0.00000000e+00  0.00000000e+00  7.69737616e-16  1.53947523e-15
   3.07895046e-15 -7.69737616e-16 -1.53947523e-15 -3.07895046e-15]
 [ 0.00000000e+00  0.00000000e+00  7.94126888e-15  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.53947523e-15  3.07895046e-15
   6.15790093e-15 -1.53947523e-15 -3.07895046e-15 -6.15790093e-15]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.24269268e-17
   1.55431223e-15  3.10862447e-1

In [58]:
linear = {('q01','q01'):8.19583044029764e-16, ('q02','q02'):2.416322205297138e-15, ('q03','q03'):7.941268879544714e-15, ('q04','q04'):-4.2426926792154264e-17, ('q05','q05'):6.9230226365330105e-16, ('q06','q06'):4.493228996257042e-15, ('q07','q07'):3.1068667381421163e-15, ('q08','q08'):7.32395650090939e-15, ('q09','q09'):1.90888051003194e-14, ('q10','q10'):-1.9966437135169598e-15, ('q11','q11'):-2.8830644024087623e-15, ('q12','q12'):-1.3252367063168984e-15}

quadratic = {('q01','q02'):1.5543122344752196e-15, ('q01','q03'):3.108624468950439e-15, ('q01','q07'):3.84868807969556e-16, ('q01','q08'):7.69737615939112e-16, ('q01','q09'):1.539475231878224e-15, ('q01','q10'):-3.84868807969556e-16, ('q01','q11'):-7.69737615939112e-16, ('q01','q12'):-1.539475231878224e-15, ('q02','q03'):6.217248937900878e-15, ('q02','q07'):7.69737615939112e-16, ('q02','q08'):1.539475231878224e-15, ('q02','q09'):3.078950463756448e-15, ('q02','q10'):-7.69737615939112e-16, ('q02','q11'):-1.539475231878224e-15, ('q02','q12'):-3.078950463756448e-15, ('q03','q07'):1.539475231878224e-15, ('q03','q08'):3.078950463756448e-15, ('q03','q09'):6.157900927512896e-15, ('q03','q10'):-1.539475231878224e-15, ('q03','q11'):-3.078950463756448e-15, ('q03','q12'):-6.157900927512896e-15, ('q04','q05'):1.5543122344752196e-15, ('q04','q06'):3.108624468950439e-15, ('q04','q07'):-3.84868807969556e-16, ('q04','q08'):-7.69737615939112e-16, ('q04','q09'):-1.539475231878224e-15, ('q04','q10'):3.84868807969556e-16, ('q04','q11'):7.69737615939112e-16, ('q04','q12'):1.539475231878224e-15, ('q05','q06'):6.217248937900878e-15, ('q05','q07'):-7.69737615939112e-16, ('q05','q08'):-1.539475231878224e-15, ('q05','q09'):-3.078950463756448e-15, ('q05','q10'):7.69737615939112e-16, ('q05','q11'):1.539475231878224e-15, ('q05','q12'):3.078950463756448e-15, ('q06','q07'):-1.539475231878224e-15, ('q06','q08'):-3.078950463756448e-15, ('q06','q09'):-6.157900927512896e-15, ('q06','q10'):1.539475231878224e-15, ('q06','q11'):3.078950463756448e-15, ('q06','q12'):6.157900927512896e-15, ('q07','q08'):2.220446049250313e-15, ('q07','q09'):4.440892098500626e-15, ('q08','q09'):8.881784197001252e-15, ('q10','q11'):2.220446049250313e-15, ('q10','q12'):4.440892098500626e-15, ('q11','q12'):8.881784197001252e-15}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12 energy num_oc. chain_b.
0    1   0   0   0   0   0   0   0   0   1   1   0   -0.0     148      0.0
1    0   0   0   0   0   0   0   0   0   0   1   0   -0.0      64      0.0
2    1   0   0   0   0   0   0   0   0   0   1   0   -0.0      79      0.0
3    0   0   0   0   0   0   0   0   0   1   1   0   -0.0      36      0.0
53   0   0   0   0   0   0   0   0   0   1   1   0   -0.0       1 0.083333
4    0   1   0   0   0   0   0   0   0   1   1   0   -0.0     207      0.0
5    0   0   0   1   0   0   0   0   0   0   1   0   -0.0      24      0.0
6    1   0   0   1   0   0   0   0   0   0   1   0   -0.0      26      0.0
7    1   0   0   0   0   0   0   0   0   0   0   1   -0.0      19      0.0
8    0   1   0   0   0   0   0   0   0   0   1   0   -0.0      50      0.0
9    0   0   0   0   0   0   0   0   0   1   0   0   -0.0      16      0.0
10   0   1   0   0   0   0   0   0   0   0   0   1   -0.0      85      0.0
11   1   0   0   1   0   

In [59]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -27
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727814912796E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185140967369080E+01
Previous error of x is -2.20097717829048633575E-09
Previous error of y is 3.40729400249983882532E-09


# Matrix Q is
[[ 4.42153135e-16  1.55431223e-15  3.10862447e-15  0.00000000e+00
   0.00000000e+00  0.00000000e+00  3.84868808e-16  7.69737616e-16
   1.53947523e-15 -3.84868808e-16 -7.69737616e-16 -1.53947523e-15]
 [ 0.00000000e+00  1.66146239e-15  6.21724894e-15  0.00000000e+00
   0.00000000e+00  0.00000000e+00  7.69737616e-16  1.53947523e-15
   3.07895046e-15 -7.69737616e-16 -1.53947523e-15 -3.07895046e-15]
 [ 0.00000000e+00  0.00000000e+00  6.43154924e-15  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.53947523e-15  3.07895046e-15
   6.15790093e-15 -1.53947523e-15 -3.07895046e-15 -6.15790093e-15]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.35002983e-16
   1.55431223e-15  3.10862447e-1

In [60]:
linear = {('q01','q01'):4.421531346284192e-16, ('q02','q02'):1.661462386494448e-15, ('q03','q03'):6.431549241939335e-15, ('q04','q04'):3.3500298260919055e-16, ('q05','q05'):1.4471620824559909e-15, ('q06','q06'):6.0029486338624205e-15, ('q07','q07'):1.6104990800533558e-16, ('q08','q08'):1.4323228406358277e-15, ('q09','q09'):7.305537779772282e-15, ('q10','q10'):9.49173116619821e-16, ('q11','q11'):3.0085692578647985e-15, ('q12','q12'):1.0458030614230222e-14}

quadratic = {('q01','q02'):1.5543122344752196e-15, ('q01','q03'):3.108624468950439e-15, ('q01','q07'):3.84868807969556e-16, ('q01','q08'):7.69737615939112e-16, ('q01','q09'):1.539475231878224e-15, ('q01','q10'):-3.84868807969556e-16, ('q01','q11'):-7.69737615939112e-16, ('q01','q12'):-1.539475231878224e-15, ('q02','q03'):6.217248937900878e-15, ('q02','q07'):7.69737615939112e-16, ('q02','q08'):1.539475231878224e-15, ('q02','q09'):3.078950463756448e-15, ('q02','q10'):-7.69737615939112e-16, ('q02','q11'):-1.539475231878224e-15, ('q02','q12'):-3.078950463756448e-15, ('q03','q07'):1.539475231878224e-15, ('q03','q08'):3.078950463756448e-15, ('q03','q09'):6.157900927512896e-15, ('q03','q10'):-1.539475231878224e-15, ('q03','q11'):-3.078950463756448e-15, ('q03','q12'):-6.157900927512896e-15, ('q04','q05'):1.5543122344752196e-15, ('q04','q06'):3.108624468950439e-15, ('q04','q07'):-3.84868807969556e-16, ('q04','q08'):-7.69737615939112e-16, ('q04','q09'):-1.539475231878224e-15, ('q04','q10'):3.84868807969556e-16, ('q04','q11'):7.69737615939112e-16, ('q04','q12'):1.539475231878224e-15, ('q05','q06'):6.217248937900878e-15, ('q05','q07'):-7.69737615939112e-16, ('q05','q08'):-1.539475231878224e-15, ('q05','q09'):-3.078950463756448e-15, ('q05','q10'):7.69737615939112e-16, ('q05','q11'):1.539475231878224e-15, ('q05','q12'):3.078950463756448e-15, ('q06','q07'):-1.539475231878224e-15, ('q06','q08'):-3.078950463756448e-15, ('q06','q09'):-6.157900927512896e-15, ('q06','q10'):1.539475231878224e-15, ('q06','q11'):3.078950463756448e-15, ('q06','q12'):6.157900927512896e-15, ('q07','q08'):2.220446049250313e-15, ('q07','q09'):4.440892098500626e-15, ('q08','q09'):8.881784197001252e-15, ('q10','q11'):2.220446049250313e-15, ('q10','q12'):4.440892098500626e-15, ('q11','q12'):8.881784197001252e-15}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12 energy num_oc. chain_b.
0    0   0   0   0   0   0   0   0   0   0   0   0    0.0     161      0.0
41   0   0   0   0   0   0   0   0   0   0   0   0    0.0       1 0.083333
42   0   0   0   0   0   0   0   0   0   0   0   0    0.0       1 0.083333
1    0   0   0   1   0   0   1   0   0   0   0   0    0.0      94      0.0
2    0   0   0   0   0   0   1   0   0   0   0   0    0.0     114      0.0
3    0   0   0   1   0   0   0   0   0   0   0   0    0.0      80      0.0
48   0   0   0   1   0   0   0   0   0   0   0   0    0.0       1 0.083333
4    1   0   0   0   0   0   0   0   0   0   0   0    0.0     129      0.0
5    1   0   0   1   0   0   0   0   0   0   0   0    0.0      61      0.0
50   1   0   0   1   0   0   0   0   0   0   0   0    0.0       1 0.083333
6    0   0   0   0   1   0   1   0   0   0   0   0    0.0      23      0.0
7    1   0   0   1   0   0   1   0   0   0   0   0    0.0      25      0.0
8    0   0   0   0   0   

In [61]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -30
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727814912796E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185140967369080E+01
Previous error of x is -2.20097717829048633575E-09
Previous error of y is 3.40729400249983882532E-09


# Matrix Q is
[[ 1.27684167e-17  2.42861287e-17  4.85722573e-17  0.00000000e+00
   0.00000000e+00  0.00000000e+00  6.01357512e-18  1.20271502e-17
   2.40543005e-17 -6.01357512e-18 -1.20271502e-17 -2.40543005e-17]
 [ 0.00000000e+00  3.76798977e-17  9.71445147e-17  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.20271502e-17  2.40543005e-17
   4.81086010e-17 -1.20271502e-17 -2.40543005e-17 -4.81086010e-17]
 [ 0.00000000e+00  0.00000000e+00  1.23932053e-16  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.40543005e-17  4.81086010e-17
   9.62172020e-17 -2.40543005e-17 -4.81086010e-17 -9.62172020e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -6.25352335e-19
   2.42861287e-17  4.85722573e-1

In [62]:
linear = {('q01','q01'):1.2768416667120614e-17, ('q02','q02'):3.7679897666078877e-17, ('q03','q03'):1.2393205265950836e-16, ('q04','q04'):-6.253523352829609e-19, ('q05','q05'):1.0892359661271731e-17, ('q06','q06'):7.035697664989408e-17, ('q07','q07'):-4.0584083158521303e-17, ('q08','q08'):-6.382093155727454e-17, ('q09','q09'):-5.825292407547679e-17, ('q10','q10'):5.793131791828937e-17, ('q11','q11'):1.3320987059634682e-16, ('q12','q12'):3.3580868023176597e-16}

quadratic = {('q01','q02'):2.4286128663675305e-17, ('q01','q03'):4.857225732735061e-17, ('q01','q07'):6.013575124524312e-18, ('q01','q08'):1.2027150249048625e-17, ('q01','q09'):2.405430049809725e-17, ('q01','q10'):-6.013575124524312e-18, ('q01','q11'):-1.2027150249048625e-17, ('q01','q12'):-2.405430049809725e-17, ('q02','q03'):9.714451465470122e-17, ('q02','q07'):1.2027150249048625e-17, ('q02','q08'):2.405430049809725e-17, ('q02','q09'):4.81086009961945e-17, ('q02','q10'):-1.2027150249048625e-17, ('q02','q11'):-2.405430049809725e-17, ('q02','q12'):-4.81086009961945e-17, ('q03','q07'):2.405430049809725e-17, ('q03','q08'):4.81086009961945e-17, ('q03','q09'):9.6217201992389e-17, ('q03','q10'):-2.405430049809725e-17, ('q03','q11'):-4.81086009961945e-17, ('q03','q12'):-9.6217201992389e-17, ('q04','q05'):2.4286128663675305e-17, ('q04','q06'):4.857225732735061e-17, ('q04','q07'):-6.013575124524312e-18, ('q04','q08'):-1.2027150249048625e-17, ('q04','q09'):-2.405430049809725e-17, ('q04','q10'):6.013575124524312e-18, ('q04','q11'):1.2027150249048625e-17, ('q04','q12'):2.405430049809725e-17, ('q05','q06'):9.714451465470122e-17, ('q05','q07'):-1.2027150249048625e-17, ('q05','q08'):-2.405430049809725e-17, ('q05','q09'):-4.81086009961945e-17, ('q05','q10'):1.2027150249048625e-17, ('q05','q11'):2.405430049809725e-17, ('q05','q12'):4.81086009961945e-17, ('q06','q07'):-2.405430049809725e-17, ('q06','q08'):-4.81086009961945e-17, ('q06','q09'):-9.6217201992389e-17, ('q06','q10'):2.405430049809725e-17, ('q06','q11'):4.81086009961945e-17, ('q06','q12'):9.6217201992389e-17, ('q07','q08'):3.469446951953614e-17, ('q07','q09'):6.938893903907228e-17, ('q08','q09'):1.3877787807814457e-16, ('q10','q11'):3.469446951953614e-17, ('q10','q12'):6.938893903907228e-17, ('q11','q12'):1.3877787807814457e-16}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12 energy num_oc. chain_b.
0    0   0   0   1   1   0   0   0   1   0   0   0   -0.0     216      0.0
49   0   0   0   1   1   0   0   0   1   0   0   0   -0.0       1 0.083333
1    0   0   0   0   1   0   0   0   1   0   0   0   -0.0      88      0.0
2    0   0   0   0   1   0   1   1   0   0   0   0   -0.0     129      0.0
18   0   0   0   0   1   0   1   1   0   0   0   0   -0.0       3 0.083333
3    0   0   0   1   1   0   1   1   0   0   0   0   -0.0     163      0.0
47   0   0   0   1   1   0   1   1   0   0   0   0   -0.0       1 0.083333
4    0   0   0   1   0   0   1   1   0   0   0   0   -0.0      91      0.0
5    0   0   0   1   1   0   1   0   1   0   0   0   -0.0      48      0.0
6    0   0   0   0   0   1   0   0   1   0   0   0   -0.0      24      0.0
7    0   0   0   1   0   0   0   0   1   0   0   0   -0.0      52      0.0
12   0   0   0   1   0   0   0   0   1   0   0   0   -0.0      10 0.083333
8    0   0   0   0   0   

In [63]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -30
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727535516024E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185103714466095E+01
Previous error of x is 5.92990545555949211121E-10
Previous error of y is -3.17996295962075237185E-10


# Matrix Q is
[[ 3.91765350e-19  2.42861287e-17  4.85722573e-17  0.00000000e+00
   0.00000000e+00  0.00000000e+00  6.01357512e-18  1.20271502e-17
   2.40543005e-17 -6.01357512e-18 -1.20271502e-17 -2.40543005e-17]
 [ 0.00000000e+00  1.29265950e-17  9.71445147e-17  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.20271502e-17  2.40543005e-17
   4.81086010e-17 -1.20271502e-17 -2.40543005e-17 -4.81086010e-17]
 [ 0.00000000e+00  0.00000000e+00  7.44254474e-17  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.40543005e-17  4.81086010e-17
   9.62172020e-17 -2.40543005e-17 -4.81086010e-17 -9.62172020e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.17512990e-17
   2.42861287e-17  4.85722573e-1

In [64]:
linear = {('q01','q01'):3.917653498946483e-19, ('q02','q02'):1.2926595031626948e-17, ('q03','q03'):7.442544739060451e-17, ('q04','q04'):1.1751298981943004e-17, ('q05','q05'):3.564566229572366e-17, ('q06','q06'):1.1986358191879791e-16, ('q07','q07'):1.076814043675022e-17, ('q08','q08'):3.888351563326851e-17, ('q09','q09'):1.471559703056093e-16, ('q10','q10'):6.579094323017852e-18, ('q11','q11'):3.050542340580378e-17, ('q12','q12'):1.3039978585067984e-16}

quadratic = {('q01','q02'):2.4286128663675305e-17, ('q01','q03'):4.857225732735061e-17, ('q01','q07'):6.013575124524312e-18, ('q01','q08'):1.2027150249048625e-17, ('q01','q09'):2.405430049809725e-17, ('q01','q10'):-6.013575124524312e-18, ('q01','q11'):-1.2027150249048625e-17, ('q01','q12'):-2.405430049809725e-17, ('q02','q03'):9.714451465470122e-17, ('q02','q07'):1.2027150249048625e-17, ('q02','q08'):2.405430049809725e-17, ('q02','q09'):4.81086009961945e-17, ('q02','q10'):-1.2027150249048625e-17, ('q02','q11'):-2.405430049809725e-17, ('q02','q12'):-4.81086009961945e-17, ('q03','q07'):2.405430049809725e-17, ('q03','q08'):4.81086009961945e-17, ('q03','q09'):9.6217201992389e-17, ('q03','q10'):-2.405430049809725e-17, ('q03','q11'):-4.81086009961945e-17, ('q03','q12'):-9.6217201992389e-17, ('q04','q05'):2.4286128663675305e-17, ('q04','q06'):4.857225732735061e-17, ('q04','q07'):-6.013575124524312e-18, ('q04','q08'):-1.2027150249048625e-17, ('q04','q09'):-2.405430049809725e-17, ('q04','q10'):6.013575124524312e-18, ('q04','q11'):1.2027150249048625e-17, ('q04','q12'):2.405430049809725e-17, ('q05','q06'):9.714451465470122e-17, ('q05','q07'):-1.2027150249048625e-17, ('q05','q08'):-2.405430049809725e-17, ('q05','q09'):-4.81086009961945e-17, ('q05','q10'):1.2027150249048625e-17, ('q05','q11'):2.405430049809725e-17, ('q05','q12'):4.81086009961945e-17, ('q06','q07'):-2.405430049809725e-17, ('q06','q08'):-4.81086009961945e-17, ('q06','q09'):-9.6217201992389e-17, ('q06','q10'):2.405430049809725e-17, ('q06','q11'):4.81086009961945e-17, ('q06','q12'):9.6217201992389e-17, ('q07','q08'):3.469446951953614e-17, ('q07','q09'):6.938893903907228e-17, ('q08','q09'):1.3877787807814457e-16, ('q10','q11'):3.469446951953614e-17, ('q10','q12'):6.938893903907228e-17, ('q11','q12'):1.3877787807814457e-16}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12 energy num_oc. chain_b.
0    0   0   0   0   0   0   0   0   0   0   0   0    0.0      68      0.0
1    1   0   0   0   0   0   0   0   0   0   0   0    0.0      94      0.0
60   1   0   0   0   0   0   0   0   0   0   0   0    0.0       1 0.083333
2    1   0   0   0   0   0   0   0   0   1   0   0    0.0     134      0.0
3    0   0   0   0   0   0   0   0   0   1   0   0    0.0      42      0.0
4    0   1   0   0   0   0   0   0   0   1   0   0    0.0      59      0.0
58   0   1   0   0   0   0   0   0   0   1   0   0    0.0       1 0.083333
5    0   0   0   0   0   0   1   0   0   0   0   0    0.0      73      0.0
6    0   0   0   1   0   0   0   0   0   0   0   0    0.0      13      0.0
7    1   0   0   1   0   0   0   0   0   0   0   0    0.0      26      0.0
8    0   1   0   0   0   0   0   0   0   0   0   0    0.0      20      0.0
9    0   0   0   1   0   0   1   0   0   0   0   0    0.0      38      0.0
10   1   0   0   0   0   

In [65]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -33
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727535516024E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185103714466095E+01
Previous error of x is 5.92990545555949211121E-10
Previous error of y is -3.17996295962075237185E-10


# Matrix Q is
[[-6.15103162e-19  3.79470760e-19  7.58941521e-19  0.00000000e+00
   0.00000000e+00  0.00000000e+00  9.39621113e-20  1.87924223e-19
   3.75848445e-19 -9.39621113e-20 -1.87924223e-19 -3.75848445e-19]
 [ 0.00000000e+00 -1.04047094e-18  1.51788304e-18  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.87924223e-19  3.75848445e-19
   7.51696891e-19 -1.87924223e-19 -3.75848445e-19 -7.51696891e-19]
 [ 0.00000000e+00  0.00000000e+00 -1.32200037e-18  0.00000000e+00
   0.00000000e+00  0.00000000e+00  3.75848445e-19  7.51696891e-19
   1.50339378e-18 -3.75848445e-19 -7.51696891e-19 -1.50339378e-18]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  8.04838542e-19
   3.79470760e-19  7.58941521e-1

In [66]:
linear = {('q01','q01'):-6.151031619105405e-19, ('q02','q02'):-1.0404709436361178e-18, ('q03','q03'):-1.3220003665323824e-18, ('q04','q04'):8.048385420955038e-19, ('q05','q05'):1.799412464375971e-18, ('q06','q06'):4.357766449491795e-18, ('q07','q07'):3.9734065366896107e-19, ('q08','q08'):1.0657318504592983e-18, ('q09','q09'):3.2156658734041013e-18, ('q10','q10'):-1.2629011054758496e-19, ('q11','q11'):1.8470322026206184e-20, ('q12','q12'):1.121142816537917e-18}

quadratic = {('q01','q02'):3.7947076036992665e-19, ('q01','q03'):7.589415207398533e-19, ('q01','q07'):9.396211132069238e-20, ('q01','q08'):1.8792422264138476e-19, ('q01','q09'):3.758484452827695e-19, ('q01','q10'):-9.396211132069238e-20, ('q01','q11'):-1.8792422264138476e-19, ('q01','q12'):-3.758484452827695e-19, ('q02','q03'):1.5178830414797066e-18, ('q02','q07'):1.8792422264138476e-19, ('q02','q08'):3.758484452827695e-19, ('q02','q09'):7.51696890565539e-19, ('q02','q10'):-1.8792422264138476e-19, ('q02','q11'):-3.758484452827695e-19, ('q02','q12'):-7.51696890565539e-19, ('q03','q07'):3.758484452827695e-19, ('q03','q08'):7.51696890565539e-19, ('q03','q09'):1.503393781131078e-18, ('q03','q10'):-3.758484452827695e-19, ('q03','q11'):-7.51696890565539e-19, ('q03','q12'):-1.503393781131078e-18, ('q04','q05'):3.7947076036992665e-19, ('q04','q06'):7.589415207398533e-19, ('q04','q07'):-9.396211132069238e-20, ('q04','q08'):-1.8792422264138476e-19, ('q04','q09'):-3.758484452827695e-19, ('q04','q10'):9.396211132069238e-20, ('q04','q11'):1.8792422264138476e-19, ('q04','q12'):3.758484452827695e-19, ('q05','q06'):1.5178830414797066e-18, ('q05','q07'):-1.8792422264138476e-19, ('q05','q08'):-3.758484452827695e-19, ('q05','q09'):-7.51696890565539e-19, ('q05','q10'):1.8792422264138476e-19, ('q05','q11'):3.758484452827695e-19, ('q05','q12'):7.51696890565539e-19, ('q06','q07'):-3.758484452827695e-19, ('q06','q08'):-7.51696890565539e-19, ('q06','q09'):-1.503393781131078e-18, ('q06','q10'):3.758484452827695e-19, ('q06','q11'):7.51696890565539e-19, ('q06','q12'):1.503393781131078e-18, ('q07','q08'):5.421010862427522e-19, ('q07','q09'):1.0842021724855044e-18, ('q08','q09'):2.168404344971009e-18, ('q10','q11'):5.421010862427522e-19, ('q10','q12'):1.0842021724855044e-18, ('q11','q12'):2.168404344971009e-18}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12 energy num_oc. chain_b.
0    1   0   1   0   0   0   0   0   0   1   1   0   -0.0     205      0.0
37   1   0   1   0   0   0   0   0   0   1   1   0   -0.0       1 0.083333
1    0   1   1   0   0   0   0   0   0   1   1   0   -0.0     231      0.0
42   0   1   1   0   0   0   0   0   0   1   1   0   -0.0       1 0.083333
2    1   0   1   0   0   0   0   0   0   0   1   0   -0.0     119      0.0
3    0   0   1   0   0   0   0   0   0   0   1   0   -0.0      99      0.0
4    0   0   1   0   0   0   0   0   0   1   1   0   -0.0      57      0.0
5    0   1   1   0   0   0   0   0   0   0   0   1   -0.0      42      0.0
41   0   1   1   0   0   0   0   0   0   0   0   1   -0.0       1 0.083333
6    0   1   1   0   0   0   0   0   0   0   1   0   -0.0      72      0.0
7    1   0   1   0   0   0   0   0   0   0   0   1   -0.0      20      0.0
8    1   1   1   0   0   0   0   0   0   1   1   0   -0.0      33      0.0
9    1   1   1   0   0   

In [67]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -33
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727593723685E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185107206925750E+01
Previous error of x is 1.09139364212751388550E-11
Previous error of y is 3.12496695187292061746E-11


# Matrix Q is
[[ 5.15718018e-20  3.79470760e-19  7.58941521e-19  0.00000000e+00
   0.00000000e+00  0.00000000e+00  9.39621113e-20  1.87924223e-19
   3.75848445e-19 -9.39621113e-20 -1.87924223e-19 -3.75848445e-19]
 [ 0.00000000e+00  2.92878984e-19  1.51788304e-18  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.87924223e-19  3.75848445e-19
   7.51696891e-19 -1.87924223e-19 -3.75848445e-19 -7.51696891e-19]
 [ 0.00000000e+00  0.00000000e+00  1.34469949e-18  0.00000000e+00
   0.00000000e+00  0.00000000e+00  3.75848445e-19  7.51696891e-19
   1.50339378e-18 -3.75848445e-19 -7.51696891e-19 -1.50339378e-18]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.38163578e-19
   3.79470760e-19  7.58941521e-19

In [68]:
linear = {('q01','q01'):5.1571801798555477e-20, ('q02','q02'):2.928789837820743e-19, ('q03','q03'):1.3446994883040019e-18, ('q04','q04'):1.3816357838640782e-19, ('q05','q05'):4.66062536957779e-19, ('q06','q06'):1.6910665946554113e-18, ('q07','q07'):5.401004185119825e-20, ('q08','q08'):3.790706268237726e-19, ('q09','q09'):1.84234342613305e-18, ('q10','q10'):2.1704050127017786e-19, ('q11','q11'):7.051315456617318e-19, ('q12','q12'):2.4944652638089683e-18}

quadratic = {('q01','q02'):3.7947076036992665e-19, ('q01','q03'):7.589415207398533e-19, ('q01','q07'):9.396211132069238e-20, ('q01','q08'):1.8792422264138476e-19, ('q01','q09'):3.758484452827695e-19, ('q01','q10'):-9.396211132069238e-20, ('q01','q11'):-1.8792422264138476e-19, ('q01','q12'):-3.758484452827695e-19, ('q02','q03'):1.5178830414797066e-18, ('q02','q07'):1.8792422264138476e-19, ('q02','q08'):3.758484452827695e-19, ('q02','q09'):7.51696890565539e-19, ('q02','q10'):-1.8792422264138476e-19, ('q02','q11'):-3.758484452827695e-19, ('q02','q12'):-7.51696890565539e-19, ('q03','q07'):3.758484452827695e-19, ('q03','q08'):7.51696890565539e-19, ('q03','q09'):1.503393781131078e-18, ('q03','q10'):-3.758484452827695e-19, ('q03','q11'):-7.51696890565539e-19, ('q03','q12'):-1.503393781131078e-18, ('q04','q05'):3.7947076036992665e-19, ('q04','q06'):7.589415207398533e-19, ('q04','q07'):-9.396211132069238e-20, ('q04','q08'):-1.8792422264138476e-19, ('q04','q09'):-3.758484452827695e-19, ('q04','q10'):9.396211132069238e-20, ('q04','q11'):1.8792422264138476e-19, ('q04','q12'):3.758484452827695e-19, ('q05','q06'):1.5178830414797066e-18, ('q05','q07'):-1.8792422264138476e-19, ('q05','q08'):-3.758484452827695e-19, ('q05','q09'):-7.51696890565539e-19, ('q05','q10'):1.8792422264138476e-19, ('q05','q11'):3.758484452827695e-19, ('q05','q12'):7.51696890565539e-19, ('q06','q07'):-3.758484452827695e-19, ('q06','q08'):-7.51696890565539e-19, ('q06','q09'):-1.503393781131078e-18, ('q06','q10'):3.758484452827695e-19, ('q06','q11'):7.51696890565539e-19, ('q06','q12'):1.503393781131078e-18, ('q07','q08'):5.421010862427522e-19, ('q07','q09'):1.0842021724855044e-18, ('q08','q09'):2.168404344971009e-18, ('q10','q11'):5.421010862427522e-19, ('q10','q12'):1.0842021724855044e-18, ('q11','q12'):2.168404344971009e-18}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12 energy num_oc. chain_b.
0    0   0   0   0   0   0   0   0   0   0   0   0    0.0      85      0.0
1    1   0   0   0   0   0   0   0   0   0   0   0    0.0      85      0.0
2    0   0   0   0   0   0   1   0   0   0   0   0    0.0     132      0.0
3    0   0   0   1   0   0   1   0   0   0   0   0    0.0     136      0.0
4    0   0   0   1   0   0   0   0   0   0   0   0    0.0      27      0.0
5    1   0   0   0   0   0   0   0   0   1   0   0    0.0      27      0.0
6    1   0   0   1   0   0   0   0   0   0   0   0    0.0      53      0.0
7    1   0   0   0   0   0   1   0   0   0   0   0    0.0      34      0.0
8    0   0   0   0   0   0   0   0   0   1   0   0    0.0      18      0.0
9    1   0   0   1   0   0   1   0   0   0   0   0    0.0      51      0.0
10   0   0   0   0   0   0   1   0   0   1   0   0    0.0      23      0.0
11   0   1   0   0   0   0   0   0   0   0   0   0    0.0      10      0.0
12   0   1   0   0   0   

In [69]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -36
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727593723685E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185107206925750E+01
Previous error of x is 1.09139364212751388550E-11
Previous error of y is 3.12496695187292061746E-11


# Matrix Q is
[[-3.92967838e-21  5.92923063e-21  1.18584613e-20  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.46815799e-21  2.93631598e-21
   5.87263196e-21 -1.46815799e-21 -2.93631598e-21 -5.87263196e-21]
 [ 0.00000000e+00 -4.89474144e-21  2.37169225e-20  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.93631598e-21  5.87263196e-21
   1.17452639e-20 -2.93631598e-21 -5.87263196e-21 -1.17452639e-20]
 [ 0.00000000e+00  0.00000000e+00  2.06897838e-21  0.00000000e+00
   0.00000000e+00  0.00000000e+00  5.87263196e-21  1.17452639e-20
   2.34905278e-20 -5.87263196e-21 -1.17452639e-20 -2.34905278e-20]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.89429369e-21
   5.92923063e-21  1.18584613e-20

In [70]:
linear = {('q01','q01'):-3.929678379045746e-21, ('q02','q02'):-4.894741442701441e-21, ('q03','q03'):2.068978376157323e-21, ('q04','q04'):6.894293694435799e-21, ('q05','q05'):1.675320270426165e-20, ('q06','q06'):4.53648666700835e-20, ('q07','q07'):-8.071821345550474e-21, ('q08','q08'):-1.1908477954829447e-20, ('q09','q09'):-6.876296964572888e-21, ('q10','q10'):1.2306986081821976e-20, ('q11','q11'):2.8849136899915454e-20, ('q12','q12'):7.463893274491691e-20}

quadratic = {('q01','q02'):5.929230630780104e-21, ('q01','q03'):1.1858461261560208e-20, ('q01','q07'):1.4681579893858185e-21, ('q01','q08'):2.936315978771637e-21, ('q01','q09'):5.872631957543274e-21, ('q01','q10'):-1.4681579893858185e-21, ('q01','q11'):-2.936315978771637e-21, ('q01','q12'):-5.872631957543274e-21, ('q02','q03'):2.3716922523120416e-20, ('q02','q07'):2.936315978771637e-21, ('q02','q08'):5.872631957543274e-21, ('q02','q09'):1.1745263915086548e-20, ('q02','q10'):-2.936315978771637e-21, ('q02','q11'):-5.872631957543274e-21, ('q02','q12'):-1.1745263915086548e-20, ('q03','q07'):5.872631957543274e-21, ('q03','q08'):1.1745263915086548e-20, ('q03','q09'):2.3490527830173095e-20, ('q03','q10'):-5.872631957543274e-21, ('q03','q11'):-1.1745263915086548e-20, ('q03','q12'):-2.3490527830173095e-20, ('q04','q05'):5.929230630780104e-21, ('q04','q06'):1.1858461261560208e-20, ('q04','q07'):-1.4681579893858185e-21, ('q04','q08'):-2.936315978771637e-21, ('q04','q09'):-5.872631957543274e-21, ('q04','q10'):1.4681579893858185e-21, ('q04','q11'):2.936315978771637e-21, ('q04','q12'):5.872631957543274e-21, ('q05','q06'):2.3716922523120416e-20, ('q05','q07'):-2.936315978771637e-21, ('q05','q08'):-5.872631957543274e-21, ('q05','q09'):-1.1745263915086548e-20, ('q05','q10'):2.936315978771637e-21, ('q05','q11'):5.872631957543274e-21, ('q05','q12'):1.1745263915086548e-20, ('q06','q07'):-5.872631957543274e-21, ('q06','q08'):-1.1745263915086548e-20, ('q06','q09'):-2.3490527830173095e-20, ('q06','q10'):5.872631957543274e-21, ('q06','q11'):1.1745263915086548e-20, ('q06','q12'):2.3490527830173095e-20, ('q07','q08'):8.470329472543003e-21, ('q07','q09'):1.6940658945086007e-20, ('q08','q09'):3.3881317890172014e-20, ('q10','q11'):8.470329472543003e-21, ('q10','q12'):1.6940658945086007e-20, ('q11','q12'):3.3881317890172014e-20}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12 energy num_oc. chain_b.
0    1   0   0   0   0   0   0   1   0   0   0   0   -0.0     205      0.0
1    0   0   0   0   0   0   0   1   0   0   0   0   -0.0      58      0.0
2    0   0   0   0   0   0   1   1   0   0   0   0   -0.0      43      0.0
3    1   0   0   0   0   0   1   1   0   0   0   0   -0.0      81      0.0
4    0   1   0   0   0   0   0   1   0   0   0   0   -0.0      39      0.0
5    1   0   0   0   0   0   1   0   0   0   0   0   -0.0      44      0.0
6    0   1   0   0   0   0   1   0   0   0   0   0   -0.0      12      0.0
7    0   0   0   1   0   0   1   1   0   0   0   0   -0.0      44      0.0
56   0   0   0   1   0   0   1   1   0   0   0   0   -0.0       1 0.083333
8    1   0   0   1   0   0   0   1   0   0   0   0   -0.0     135      0.0
9    1   0   0   1   0   0   1   1   0   0   0   0   -0.0      97      0.0
10   0   0   0   0   0   0   1   0   0   0   0   0   -0.0       4      0.0
11   0   0   0   1   0   

In [71]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -36
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727595178876E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185106915887445E+01
Previous error of x is -3.63797880709171295166E-12
Previous error of y is 2.14583906199550256133E-12


# Matrix Q is
[[ 2.03421155e-21  5.92923063e-21  1.18584613e-20  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.46815799e-21  2.93631598e-21
   5.87263196e-21 -1.46815799e-21 -2.93631598e-21 -5.87263196e-21]
 [ 0.00000000e+00  7.03303842e-21  2.37169225e-20  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.93631598e-21  5.87263196e-21
   1.17452639e-20 -2.93631598e-21 -5.87263196e-21 -1.17452639e-20]
 [ 0.00000000e+00  0.00000000e+00  2.59245381e-20  0.00000000e+00
   0.00000000e+00  0.00000000e+00  5.87263196e-21  1.17452639e-20
   2.34905278e-20 -5.87263196e-21 -1.17452639e-20 -2.34905278e-20]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  9.30403765e-22
   5.92923063e-21  1.18584613e-2

In [72]:
linear = {('q01','q01'):2.034211550368985e-21, ('q02','q02'):7.033038416128022e-21, ('q03','q03'):2.5924538093816252e-20, ('q04','q04'):9.304037650210668e-22, ('q05','q05'):4.825422845432186e-21, ('q06','q06'):2.150930695242458e-20, ('q07','q07'):1.869209699037199e-21, ('q08','q08'):7.9735841343459e-21, ('q09','q09'):3.2887827213777807e-20, ('q10','q10'):2.3659550372343026e-21, ('q11','q11'):8.967074810740108e-21, ('q12','q12'):3.4874808566566226e-20}

quadratic = {('q01','q02'):5.929230630780104e-21, ('q01','q03'):1.1858461261560208e-20, ('q01','q07'):1.4681579893858185e-21, ('q01','q08'):2.936315978771637e-21, ('q01','q09'):5.872631957543274e-21, ('q01','q10'):-1.4681579893858185e-21, ('q01','q11'):-2.936315978771637e-21, ('q01','q12'):-5.872631957543274e-21, ('q02','q03'):2.3716922523120416e-20, ('q02','q07'):2.936315978771637e-21, ('q02','q08'):5.872631957543274e-21, ('q02','q09'):1.1745263915086548e-20, ('q02','q10'):-2.936315978771637e-21, ('q02','q11'):-5.872631957543274e-21, ('q02','q12'):-1.1745263915086548e-20, ('q03','q07'):5.872631957543274e-21, ('q03','q08'):1.1745263915086548e-20, ('q03','q09'):2.3490527830173095e-20, ('q03','q10'):-5.872631957543274e-21, ('q03','q11'):-1.1745263915086548e-20, ('q03','q12'):-2.3490527830173095e-20, ('q04','q05'):5.929230630780104e-21, ('q04','q06'):1.1858461261560208e-20, ('q04','q07'):-1.4681579893858185e-21, ('q04','q08'):-2.936315978771637e-21, ('q04','q09'):-5.872631957543274e-21, ('q04','q10'):1.4681579893858185e-21, ('q04','q11'):2.936315978771637e-21, ('q04','q12'):5.872631957543274e-21, ('q05','q06'):2.3716922523120416e-20, ('q05','q07'):-2.936315978771637e-21, ('q05','q08'):-5.872631957543274e-21, ('q05','q09'):-1.1745263915086548e-20, ('q05','q10'):2.936315978771637e-21, ('q05','q11'):5.872631957543274e-21, ('q05','q12'):1.1745263915086548e-20, ('q06','q07'):-5.872631957543274e-21, ('q06','q08'):-1.1745263915086548e-20, ('q06','q09'):-2.3490527830173095e-20, ('q06','q10'):5.872631957543274e-21, ('q06','q11'):1.1745263915086548e-20, ('q06','q12'):2.3490527830173095e-20, ('q07','q08'):8.470329472543003e-21, ('q07','q09'):1.6940658945086007e-20, ('q08','q09'):3.3881317890172014e-20, ('q10','q11'):8.470329472543003e-21, ('q10','q12'):1.6940658945086007e-20, ('q11','q12'):3.3881317890172014e-20}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12 energy num_oc. chain_b.
0    0   0   0   0   0   0   0   0   0   0   0   0    0.0     141      0.0
46   0   0   0   0   0   0   0   0   0   0   0   0    0.0       1 0.083333
1    0   0   0   1   0   0   0   0   0   0   0   0    0.0      61      0.0
2    0   0   0   1   0   0   1   0   0   0   0   0    0.0      38      0.0
3    0   0   0   0   0   0   1   0   0   0   0   0    0.0      69      0.0
4    1   0   0   0   0   0   0   0   0   0   0   0    0.0      78      0.0
5    0   0   0   0   0   0   0   0   0   1   0   0    0.0      48      0.0
6    1   0   0   0   0   0   0   0   0   1   0   0    0.0      61      0.0
68   1   0   0   0   0   0   0   0   0   1   0   0    0.0       1 0.083333
7    1   0   0   1   0   0   0   0   0   0   0   0    0.0      15      0.0
8    0   0   0   0   1   0   1   0   0   0   0   0    0.0      47      0.0
9    0   0   0   0   0   0   1   0   0   1   0   0    0.0      30      0.0
10   0   0   0   1   0   

In [73]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -39
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727595178876E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185106915887445E+01
Previous error of x is -3.63797880709171295166E-12
Previous error of y is 2.14583906199550256133E-12


# Matrix Q is
[[ 9.21490437e-23  9.26442286e-23  1.85288457e-22  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.29399686e-23  4.58799372e-23
   9.17598743e-23 -2.29399686e-23 -4.58799372e-23 -9.17598743e-23]
 [ 0.00000000e+00  2.30620202e-22  3.70576914e-22  0.00000000e+00
   0.00000000e+00  0.00000000e+00  4.58799372e-23  9.17598743e-23
   1.83519749e-22 -4.58799372e-23 -9.17598743e-23 -1.83519749e-22]
 [ 0.00000000e+00  0.00000000e+00  6.46528861e-22  0.00000000e+00
   0.00000000e+00  0.00000000e+00  9.17598743e-23  1.83519749e-22
   3.67039497e-22 -9.17598743e-23 -1.83519749e-22 -3.67039497e-22]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.58269294e-23
   9.26442286e-23  1.85288457e-2

In [74]:
linear = {('q01','q01'):9.214904373572965e-23, ('q02','q02'):2.306202017744289e-22, ('q03','q03'):6.46528860760736e-22, ('q04','q04'):-4.5826929432760104e-23, ('q05','q05'):-4.5331744562550646e-23, ('q06','q06'):9.462496808677695e-23, ('q07','q07'):2.0406408648021195e-24, ('q08','q08'):7.025573073384645e-23, ('q09','q09'):4.0520925748466176e-22, ('q10','q10'):6.41338081394401e-23, ('q11','q11'):1.944420652831224e-22, ('q12','q12'):6.535819265832138e-22}

quadratic = {('q01','q02'):9.264422860593912e-23, ('q01','q03'):1.8528845721187825e-22, ('q01','q07'):2.2939968584153413e-23, ('q01','q08'):4.5879937168306827e-23, ('q01','q09'):9.175987433661365e-23, ('q01','q10'):-2.2939968584153413e-23, ('q01','q11'):-4.5879937168306827e-23, ('q01','q12'):-9.175987433661365e-23, ('q02','q03'):3.705769144237565e-22, ('q02','q07'):4.5879937168306827e-23, ('q02','q08'):9.175987433661365e-23, ('q02','q09'):1.835197486732273e-22, ('q02','q10'):-4.5879937168306827e-23, ('q02','q11'):-9.175987433661365e-23, ('q02','q12'):-1.835197486732273e-22, ('q03','q07'):9.175987433661365e-23, ('q03','q08'):1.835197486732273e-22, ('q03','q09'):3.670394973464546e-22, ('q03','q10'):-9.175987433661365e-23, ('q03','q11'):-1.835197486732273e-22, ('q03','q12'):-3.670394973464546e-22, ('q04','q05'):9.264422860593912e-23, ('q04','q06'):1.8528845721187825e-22, ('q04','q07'):-2.2939968584153413e-23, ('q04','q08'):-4.5879937168306827e-23, ('q04','q09'):-9.175987433661365e-23, ('q04','q10'):2.2939968584153413e-23, ('q04','q11'):4.5879937168306827e-23, ('q04','q12'):9.175987433661365e-23, ('q05','q06'):3.705769144237565e-22, ('q05','q07'):-4.5879937168306827e-23, ('q05','q08'):-9.175987433661365e-23, ('q05','q09'):-1.835197486732273e-22, ('q05','q10'):4.5879937168306827e-23, ('q05','q11'):9.175987433661365e-23, ('q05','q12'):1.835197486732273e-22, ('q06','q07'):-9.175987433661365e-23, ('q06','q08'):-1.835197486732273e-22, ('q06','q09'):-3.670394973464546e-22, ('q06','q10'):9.175987433661365e-23, ('q06','q11'):1.835197486732273e-22, ('q06','q12'):3.670394973464546e-22, ('q07','q08'):1.3234889800848443e-22, ('q07','q09'):2.6469779601696886e-22, ('q08','q09'):5.293955920339377e-22, ('q10','q11'):1.3234889800848443e-22, ('q10','q12'):2.6469779601696886e-22, ('q11','q12'):5.293955920339377e-22}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q01 q02 q03 q04 q05 q06 q07 q08 q09 q10 q11 q12 energy num_oc. chain_b.
0    0   0   0   0   1   0   1   0   0   0   0   0   -0.0     179      0.0
13   0   0   0   0   1   0   1   0   0   0   0   0   -0.0       5 0.083333
47   0   0   0   0   1   0   1   0   0   0   0   0   -0.0       1 0.083333
1    0   0   0   0   1   0   0   1   0   0   0   0   -0.0      33      0.0
2    0   0   0   1   0   0   1   0   0   0   0   0   -0.0     242      0.0
18   0   0   0   1   0   0   1   0   0   0   0   0   -0.0       7 0.083333
20   0   0   0   1   0   0   1   0   0   0   0   0   -0.0       8 0.083333
49   0   0   0   1   0   0   1   0   0   0   0   0   -0.0       1 0.083333
3    0   0   0   1   1   0   0   1   0   0   0   0   -0.0      53      0.0
4    0   0   0   1   1   0   1   0   0   0   0   0   -0.0     168      0.0
22   0   0   0   1   1   0   1   0   0   0   0   0   -0.0       9 0.083333
48   0   0   0   1   1   0   1   0   0   0   0   0   -0.0       1 0.083333
5    0   0   0   1   0   

In [75]:
Samples = sampleset.samples()
qb_data = Samples[0,['q01','q02','q03','q04','q05','q06','q07','q08','q09','q10','q11','q12']]
q1p = pow(2,Curr_Pow)*(qb_data[0]+2*qb_data[1]+4*qb_data[2])
q1n = -pow(2,Curr_Pow)*(qb_data[3]+2*qb_data[4]+4*qb_data[5])
q2p = pow(2,Curr_Pow)*(qb_data[6]+2*qb_data[7]+4*qb_data[8])
q2n = -pow(2,Curr_Pow)*(qb_data[9]+2*qb_data[10]+4*qb_data[11])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -39
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727594815078E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185106897697551E+01
Previous error of x is 0.00000000000000000000E+00
Previous error of y is 3.26849658449646085501E-13


# Matrix Q is
[[ 1.57625288e-23  9.26442286e-23  1.85288457e-22  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.29399686e-23  4.58799372e-23
   9.17598743e-23 -2.29399686e-23 -4.58799372e-23 -9.17598743e-23]
 [ 0.00000000e+00  7.78471720e-23  3.70576914e-22  0.00000000e+00
   0.00000000e+00  0.00000000e+00  4.58799372e-23  9.17598743e-23
   1.83519749e-22 -4.58799372e-23 -9.17598743e-23 -1.83519749e-22]
 [ 0.00000000e+00  0.00000000e+00  3.40982801e-22  0.00000000e+00
   0.00000000e+00  0.00000000e+00  9.17598743e-23  1.83519749e-22
   3.67039497e-22 -9.17598743e-23 -1.83519749e-22 -3.67039497e-22]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.05595855e-23
   9.26442286e-23  1.85288457e-22